In [379]:
import pandas as pd
pd.set_option('max_colwidth', 100)
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
import itertools
import scipy.stats as st
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.feature_selection import mutual_info_classif
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
import rpy2

In [466]:
# GENERAL
#datapath = 'data/NIMH0007_genpop_num_OUTPUT.csv'
#path_to_write = 'data/mydata_1general_fordescriptive.csv'
#path_to_write_ICC = 'data/mydata_1_forICC.csv'
# HIGH RISK
datapath = 'data/NIMH0007_mental_health_num_OUTPUT.csv'
path_to_write = 'data/mydata_1highrisk_fordescriptive.csv'
path_to_write_ICC = 'data/mydata_1highrisk_forICC.csv'

In [467]:
#loads the codesheet file
# wherever the emane column exists, creates a dict variablename:ename
code_book = pd.read_csv('dylan_github/yougov_codesheet_alignment.tsv', sep='\t')
ename_lut = {vn:en for vn, en in code_book.loc[code_book.ename.notnull(), ['var_name', 'ename']].values}
ename_lut

{'caseid': 'caseid',
 'birthyr': 'birthyr',
 'gender': 'gender',
 'sex': 'sex',
 'educ': 'educ',
 'inputstate': 'inputstate',
 'sample_type': 'sample_type',
 'screener_1': 'screener_1ongoing',
 'screener_2': 'screener_2impact',
 'screener_3_1': 'screener_3depression',
 'screener_3_2': 'screener_4anxiety',
 'screener_3_3': 'screener_5attention',
 'consent': 'consent',
 'FNM_Q24': 'mood_yn',
 'FNM_Q26_d': 'mood_years',
 'FNM_Q28': 'mood_bothered',
 'FNM_Q29': 'anxiety_yn',
 'FNM_Q31_d': 'anxiety_years',
 'FNM_Q33': 'anxiety_bothered',
 'FNM_34': 'attention_yn',
 'FNM_Q36_d': 'attention_years',
 'FNM_Q38': 'attention_bothered',
 'FNM_Q1_1': 'inattention_1',
 'FNM_Q1_2': 'inattention_2',
 'FNM_Q1_3': 'inattention_3',
 'FNM_Q1_4': 'inattention_4',
 'FNM_Q1_5': 'inattention_5',
 'FNM_Q1_6': 'inattention_6',
 'FNM_Q1_7': 'inattention_7',
 'FNM_Q1_8': 'inattention_8',
 'FNM_Q1_9': 'inattention_9',
 'FNM_Q2_1': 'hyperactivity_1',
 'FNM_Q2_2': 'hyperactivity_2',
 'FNM_Q2_3': 'hyperactivity_3',
 

In [468]:
for vn, en in code_book.loc[code_book.ename.notnull(), ['var_name', 'ename']].values:
    ename_lut[vn+'_recontact'] = en+'_recontact'

ename_lut

{'caseid': 'caseid',
 'birthyr': 'birthyr',
 'gender': 'gender',
 'sex': 'sex',
 'educ': 'educ',
 'inputstate': 'inputstate',
 'sample_type': 'sample_type',
 'screener_1': 'screener_1ongoing',
 'screener_2': 'screener_2impact',
 'screener_3_1': 'screener_3depression',
 'screener_3_2': 'screener_4anxiety',
 'screener_3_3': 'screener_5attention',
 'consent': 'consent',
 'FNM_Q24': 'mood_yn',
 'FNM_Q26_d': 'mood_years',
 'FNM_Q28': 'mood_bothered',
 'FNM_Q29': 'anxiety_yn',
 'FNM_Q31_d': 'anxiety_years',
 'FNM_Q33': 'anxiety_bothered',
 'FNM_34': 'attention_yn',
 'FNM_Q36_d': 'attention_years',
 'FNM_Q38': 'attention_bothered',
 'FNM_Q1_1': 'inattention_1',
 'FNM_Q1_2': 'inattention_2',
 'FNM_Q1_3': 'inattention_3',
 'FNM_Q1_4': 'inattention_4',
 'FNM_Q1_5': 'inattention_5',
 'FNM_Q1_6': 'inattention_6',
 'FNM_Q1_7': 'inattention_7',
 'FNM_Q1_8': 'inattention_8',
 'FNM_Q1_9': 'inattention_9',
 'FNM_Q2_1': 'hyperactivity_1',
 'FNM_Q2_2': 'hyperactivity_2',
 'FNM_Q2_3': 'hyperactivity_3',
 

In [469]:
"""ename_lut['FNM_Q24_recontact'] = 'mood_yn_recontact'
ename_lut['FNM_Q26_d_recontact'] ='mood_years_recontact'
ename_lut['FNM_Q28_recontact'] ='mood_bothered_recontact'
ename_lut['FNM_Q29_recontact'] ='anxiety_yn_recontact'
ename_lut['FNM_Q31_d_recontact'] ='anxiety_years_recontact'
ename_lut['FNM_Q33_recontact'] ='anxiety_bothered_recontact'
ename_lut['FNM_34_recontact'] ='attention_yn_recontact'
ename_lut['FNM_Q36_d_recontact'] ='attention_years_recontact'
ename_lut['FNM_Q38_recontact'] ='attention_bothered_recontact'
ename_lut['FNM_Q8_5_recontact'] ='check_moderately_recontact'
ename_lut['FNM_Q22_3_recontact'] ='check_notatall_recontact'
ename_lut['FNM_Q42_m_10_recontact'] ='todaycheck_1_recontact'
ename_lut['FNM_44_m_28_recontact'] ='todaycheck_2_recontact'
    
ename_lut"""

"ename_lut['FNM_Q24_recontact'] = 'mood_yn_recontact'\nename_lut['FNM_Q26_d_recontact'] ='mood_years_recontact'\nename_lut['FNM_Q28_recontact'] ='mood_bothered_recontact'\nename_lut['FNM_Q29_recontact'] ='anxiety_yn_recontact'\nename_lut['FNM_Q31_d_recontact'] ='anxiety_years_recontact'\nename_lut['FNM_Q33_recontact'] ='anxiety_bothered_recontact'\nename_lut['FNM_34_recontact'] ='attention_yn_recontact'\nename_lut['FNM_Q36_d_recontact'] ='attention_years_recontact'\nename_lut['FNM_Q38_recontact'] ='attention_bothered_recontact'\nename_lut['FNM_Q8_5_recontact'] ='check_moderately_recontact'\nename_lut['FNM_Q22_3_recontact'] ='check_notatall_recontact'\nename_lut['FNM_Q42_m_10_recontact'] ='todaycheck_1_recontact'\nename_lut['FNM_44_m_28_recontact'] ='todaycheck_2_recontact'\n    \nename_lut"

In [470]:
"""for item in ['inattention_1', 'inattention_2', 'inattention_3', 'inattention_4', 'inattention_5', 'inattention_6', 'inattention_7', 'inattention_8', 'inattention_9',
             'hyperactivity_1', 'hyperactivity_2', 'hyperactivity_3', 'hyperactivity_4', 'hyperactivity_5',
             'impulsivity_1', 'impulsivity_2', 'impulsivity_3', 'impulsivity_4',
             'sct_1', 'sct_2', 'sct_3', 'sct_4', 'sct_5', 'sct_6', 'sct_7', 'sct_8', 'sct_9',
             'gad_1', 'gad_2', 'gad_3', 'gad_4', 'gad_5', 'gad_6', 'gad_7',
             'phq_1', 'phq_2', 'phq_3', 'phq_4', 'phq_5', 'phq_6', 'phq_7', 'phq_8',
             'hitop157', 'hitop81', 'hitop34', 'hitop54', 'hitop243', 'hitop182', 'hitop69', 'hitop89', 'hitop50', 'hitop129', 'hitop265', 'hitop124', 'hitop231', 'hitop93', 'hitop67', 'hitop245', 'hitop281', 'hitop141', 'hitop40', 'hitop204', 'hitop21', 'hitop236', 'hitop280', 'hitop84', 'hitop120', 'hitop77', 'hitop92', 'hitop258', 'hitop39', 'hitop254', 'hitop215', 'hitop95', 'hitop106', 'hitop283', 'hitop16', 'hitop20', 'hitop189', 'hitop1', 'hitop136', 'hitop246', 'hitop248', 'hitop257', 'hitop114', 'hitop117', 'hitop250', 'hitop200', 'hitop160', 'hitop23', 'hitop165', 'hitop244', 'hitop9', 'hitop142', 'hitop230', 'hitop149', 'hitop247', 'hitop99', 'hitop66', 'hitop240', 'hitop222', 'hitop90', 'hitop113', 'hitop278', 'hitop203', 'hitop159', 'hitop123', 'hitop275', 'hitop268', 'hitop225', 'hitop143', 'hitop151', 'hitop181', 'hitop211', 'hitop17', 'hitop126', 'hitop5', 'hitop261', 'hitop220', 'hitop15', 'hitop72', 'hitop140', 'hitop109', 'hitop197', 'hitop104']:
    print(item+'_recontact')"""

"for item in ['inattention_1', 'inattention_2', 'inattention_3', 'inattention_4', 'inattention_5', 'inattention_6', 'inattention_7', 'inattention_8', 'inattention_9',\n             'hyperactivity_1', 'hyperactivity_2', 'hyperactivity_3', 'hyperactivity_4', 'hyperactivity_5',\n             'impulsivity_1', 'impulsivity_2', 'impulsivity_3', 'impulsivity_4',\n             'sct_1', 'sct_2', 'sct_3', 'sct_4', 'sct_5', 'sct_6', 'sct_7', 'sct_8', 'sct_9',\n             'gad_1', 'gad_2', 'gad_3', 'gad_4', 'gad_5', 'gad_6', 'gad_7',\n             'phq_1', 'phq_2', 'phq_3', 'phq_4', 'phq_5', 'phq_6', 'phq_7', 'phq_8',\n             'hitop157', 'hitop81', 'hitop34', 'hitop54', 'hitop243', 'hitop182', 'hitop69', 'hitop89', 'hitop50', 'hitop129', 'hitop265', 'hitop124', 'hitop231', 'hitop93', 'hitop67', 'hitop245', 'hitop281', 'hitop141', 'hitop40', 'hitop204', 'hitop21', 'hitop236', 'hitop280', 'hitop84', 'hitop120', 'hitop77', 'hitop92', 'hitop258', 'hitop39', 'hitop254', 'hitop215', 'hitop95', '

In [471]:
# this groups hitop items by subscales
# marie: add the initial and the recontact scales

tmp = code_book.query('ename.notnull()')
scale_lut = {}
for ss, df in tmp.loc[tmp.ename.str.contains('hitop') & ~tmp.ename.str.contains('today')].groupby('subscale'):
    scale_name = ss.replace(" ", "_").replace("/", "_").replace("-", "_")
    items = df.ename.values
    scale_lut[scale_name] = items
    
 #   scale_name_initial = scale_name+'_initial'
 #   scale_lut[scale_name_initial] = items 
    scale_name_recontact = scale_name+'_recontact'
    items_recontact = []
    for item in items:
        items_recontact.append(item+'_recontact')
    scale_lut[scale_name_recontact] = np.array(items_recontact, dtype=object)
scale_lut

{'anhedonic_depression': array(['hitop157', 'hitop182', 'hitop93', 'hitop84', 'hitop77', 'hitop92',
        'hitop39', 'hitop246', 'hitop230', 'hitop123'], dtype=object),
 'anhedonic_depression_recontact': array(['hitop157_recontact', 'hitop182_recontact', 'hitop93_recontact',
        'hitop84_recontact', 'hitop77_recontact', 'hitop92_recontact',
        'hitop39_recontact', 'hitop246_recontact', 'hitop230_recontact',
        'hitop123_recontact'], dtype=object),
 'anxious_worry': array(['hitop34', 'hitop89', 'hitop265', 'hitop20', 'hitop248',
        'hitop240', 'hitop203'], dtype=object),
 'anxious_worry_recontact': array(['hitop34_recontact', 'hitop89_recontact', 'hitop265_recontact',
        'hitop20_recontact', 'hitop248_recontact', 'hitop240_recontact',
        'hitop203_recontact'], dtype=object),
 'appetite_gain': array(['hitop243', 'hitop141', 'hitop120', 'hitop275'], dtype=object),
 'appetite_gain_recontact': array(['hitop243_recontact', 'hitop141_recontact', 'hitop120_recont

In [472]:
scale_lut

{'anhedonic_depression': array(['hitop157', 'hitop182', 'hitop93', 'hitop84', 'hitop77', 'hitop92',
        'hitop39', 'hitop246', 'hitop230', 'hitop123'], dtype=object),
 'anhedonic_depression_recontact': array(['hitop157_recontact', 'hitop182_recontact', 'hitop93_recontact',
        'hitop84_recontact', 'hitop77_recontact', 'hitop92_recontact',
        'hitop39_recontact', 'hitop246_recontact', 'hitop230_recontact',
        'hitop123_recontact'], dtype=object),
 'anxious_worry': array(['hitop34', 'hitop89', 'hitop265', 'hitop20', 'hitop248',
        'hitop240', 'hitop203'], dtype=object),
 'anxious_worry_recontact': array(['hitop34_recontact', 'hitop89_recontact', 'hitop265_recontact',
        'hitop20_recontact', 'hitop248_recontact', 'hitop240_recontact',
        'hitop203_recontact'], dtype=object),
 'appetite_gain': array(['hitop243', 'hitop141', 'hitop120', 'hitop275'], dtype=object),
 'appetite_gain_recontact': array(['hitop243_recontact', 'hitop141_recontact', 'hitop120_recont

In [473]:
# done working with the codesheet file, now opening the actual data

In [474]:
dat = pd.read_csv(datapath, dtype={'caseid':str}, engine='python')
dat.head()

,caseid,weight,sample_type,screener_1,screener_2,screener_3_1,screener_3_2,screener_3_3,consent,FNM_Q24,FNM_Q25_1,FNM_Q25_2,FNM_Q25_3,FNM_Q25_4,FNM_Q25_5,FNM_Q25_6,FNM_Q25_955,FNM_Q25_933,FNM_Q25_open1,FNM_Q26_d,FNM_Q27_1,FNM_Q27_2,FNM_Q27_3,FNM_Q27_4,FNM_Q27_5,FNM_Q27_6,FNM_Q27_7,FNM_Q27_8,FNM_Q27_9,FNM_Q27_10,FNM_Q27_11,FNM_Q27_12,FNM_Q27_13,FNM_Q27_14,FNM_Q27_15,FNM_Q27_16,FNM_Q27_17,FNM_Q27_18,FNM_Q27_19,FNM_Q27_955,FNM_Q27_933,FNM_Q27_open1,FNM_Q28,FNM_Q29,FNM_Q30_m_1,FNM_Q30_m_2,FNM_Q30_m_3,FNM_Q30_m_4,FNM_Q30_m_5,FNM_Q30_m_6,FNM_Q30_m_7,FNM_Q30_m_8,FNM_Q30_m_955,FNM_Q30_m_933,FNM_Q30_open1,FNM_Q31_d,FNM_Q32_1,FNM_Q32_2,FNM_Q32_3,FNM_Q32_4,FNM_Q32_5,FNM_Q32_6,FNM_Q32_7,FNM_Q32_8,FNM_Q32_9,FNM_Q32_10,FNM_Q32_955,FNM_Q32_933,FNM_Q32_open1,FNM_Q33,FNM_34,FNM_Q35_m_1,FNM_Q35_m_2,FNM_Q35_m_3,FNM_Q35_m_933,FNM_Q35_open1,FNM_Q36_d,FNM_Q37_m_1,FNM_Q37_m_2,FNM_Q37_m_3,FNM_Q37_m_4,FNM_Q37_m_5,FNM_Q37_m_6,FNM_Q37_m_7,FNM_Q37_m_8,FNM_Q37_m_9,FNM_Q37_m_955,FNM_Q37_m_933,FNM_Q37_open1,FNM_Q38,FNM_Q1_1,FNM_Q1_2,FNM_Q1_3,FNM_Q1_4,FNM_Q1_5,FNM_Q1_6,FNM_Q1_attn,FNM_Q1_7,FNM_Q1_8,FNM_Q1_9,FNM_Q2_1,FNM_Q2_2,FNM_Q2_3,FNM_Q2_4,FNM_Q2_5,FNM_Q3_1,FNM_Q3_2,FNM_Q3_3,FNM_Q3_4,FNM_Q4_1,FNM_Q4_2,FNM_Q4_3,FNM_Q4_4,FNM_Q4_5,FNM_Q4_6,FNM_Q4_7,FNM_Q4_8,FNM_Q4_9,FNM_Q5_1,FNM_Q5_2,FNM_Q5_3,FNM_Q5_4,FNM_Q5_5,FNM_Q5_6,FNM_Q5_7,FNM_Q6_1,FNM_Q6_2,FNM_Q6_3,FNM_Q6_4,FNM_Q6_5,FNM_Q6_6,FNM_Q6_7,FNM_Q6_8,FNM_Q7_1,FNM_Q7_2,FNM_Q7_3,FNM_Q7_4,FNM_Q7_5,FNM_Q8_1,FNM_Q8_2,FNM_Q8_3,FNM_Q8_4,FNM_Q8_5,FNM_Q8_6,FNM_Q9_1,FNM_Q9_2,FNM_Q9_3,FNM_Q9_4,FNM_Q9_5,FNM_Q10_1,FNM_Q10_2,FNM_Q10_3,FNM_Q10_4,FNM_Q10_5,FNM_Q11_1,FNM_Q11_2,FNM_Q11_3,FNM_Q11_4,FNM_Q11_5,FNM_Q12_1,FNM_Q12_2,FNM_Q12_3,FNM_Q12_4,FNM_Q12_5,FNM_Q13_1,FNM_Q13_2,FNM_Q13_3,FNM_Q13_4,FNM_Q13_5,FNM_Q14_1,FNM_Q14_2,FNM_Q14_3,FNM_Q14_4,FNM_Q14_5,FNM_Q15_1,FNM_Q15_2,FNM_Q15_3,FNM_Q15_4,FNM_Q15_5,FNM_Q16_1,FNM_Q16_2,FNM_Q16_3,FNM_Q16_4,FNM_Q16_5,FNM_Q17_1,FNM_Q17_2,FNM_Q17_3,FNM_Q17_4,FNM_Q17_5,FNM_Q18_1,FNM_Q18_2,FNM_Q18_3,FNM_Q18_4,FNM_Q18_5,FNM_Q19_1,FNM_Q19_2,FNM_Q19_3,FNM_Q19_4,FNM_Q19_5,FNM_Q20_1,FNM_Q20_2,FNM_Q20_3,FNM_Q20_4,FNM_Q20_5,FNM_Q21_1,FNM_Q21_2,FNM_Q21_3,FNM_Q21_4,FNM_Q21_5,FNM_Q22_1,FNM_Q22_2,FNM_Q22_3,FNM_Q22_4,FNM_Q22_5,FNM_Q22_6,FNM_Q23_1,FNM_Q23_2,FNM_Q23_3,FNM_Q40_m_1,FNM_Q40_m_2,FNM_Q40_m_3,FNM_Q40_m_4,FNM_Q40_m_5,FNM_Q40_m_6,FNM_Q40_m_7,FNM_Q40_m_8,FNM_Q40_m_9,FNM_Q40_m_10,FNM_Q40_m_11,FNM_Q40_m_12,FNM_Q40_m_13,FNM_Q40_m_14,FNM_Q40_m_15,FNM_Q40_m_16,FNM_Q40_m_17,FNM_Q40_m_18,FNM_Q40_m_19,FNM_Q40_m_20,FNM_Q40_m_21,FNM_Q40_m_22,FNM_Q40_m_23,FNM_Q40_m_24,FNM_Q40_m_25,FNM_Q40_m_26,FNM_Q40_m_27,FNM_Q40_m_944,FNM_Q41_m_1,FNM_Q41_m_2,FNM_Q41_m_3,FNM_Q41_m_4,...,FNM_Q3_1_recontact,FNM_Q3_2_recontact,FNM_Q3_3_recontact,FNM_Q3_4_recontact,FNM_Q4_1_recontact,FNM_Q4_2_recontact,FNM_Q4_3_recontact,FNM_Q4_4_recontact,FNM_Q4_5_recontact,FNM_Q4_6_recontact,FNM_Q4_7_recontact,FNM_Q4_8_recontact,FNM_Q4_9_recontact,FNM_Q5_1_recontact,FNM_Q5_2_recontact,FNM_Q5_3_recontact,FNM_Q5_4_recontact,FNM_Q5_5_recontact,FNM_Q5_6_recontact,FNM_Q5_7_recontact,FNM_Q6_1_recontact,FNM_Q6_2_recontact,FNM_Q6_3_recontact,FNM_Q6_4_recontact,FNM_Q6_5_recontact,FNM_Q6_6_recontact,FNM_Q6_7_recontact,FNM_Q6_8_recontact,FNM_Q7_1_recontact,FNM_Q7_2_recontact,FNM_Q7_3_recontact,FNM_Q7_4_recontact,FNM_Q7_5_recontact,FNM_Q8_1_recontact,FNM_Q8_2_recontact,FNM_Q8_3_recontact,FNM_Q8_4_recontact,FNM_Q8_5_recontact,FNM_Q8_6_recontact,FNM_Q9_1_recontact,FNM_Q9_2_recontact,FNM_Q9_3_recontact,FNM_Q9_4_recontact,FNM_Q9_5_recontact,FNM_Q10_1_recontact,FNM_Q10_2_recontact,FNM_Q10_3_recontact,FNM_Q10_4_recontact,FNM_Q10_5_recontact,FNM_Q11_1_recontact,FNM_Q11_2_recontact,FNM_Q11_3_recontact,FNM_Q11_4_recontact,FNM_Q11_5_recontact,FNM_Q12_1_recontact,FNM_Q12_2_recontact,FNM_Q12_3_recontact,FNM_Q12_4_recontact,FNM_Q12_5_recontact,FNM_Q13_1_recontact,FNM_Q13_2_recontact,FNM_Q13_3_recontact,FNM_Q13_4_recontact,FNM_Q13_5_recontact,FNM_Q14_1_recontact,FNM_Q14_2_recontact,FNM_Q14_3_recontact,FNM_Q14_4_recontact,FNM_Q14_5_recontact,FNM_Q15_1_recontact,FNM_Q15_2_re

In [475]:
for col in dat.columns:
    print(col)

caseid
weight
sample_type
screener_1
screener_2
screener_3_1
screener_3_2
screener_3_3
consent
FNM_Q24
FNM_Q25_1
FNM_Q25_2
FNM_Q25_3
FNM_Q25_4
FNM_Q25_5
FNM_Q25_6
FNM_Q25_955
FNM_Q25_933
FNM_Q25_open1
FNM_Q26_d
FNM_Q27_1
FNM_Q27_2
FNM_Q27_3
FNM_Q27_4
FNM_Q27_5
FNM_Q27_6
FNM_Q27_7
FNM_Q27_8
FNM_Q27_9
FNM_Q27_10
FNM_Q27_11
FNM_Q27_12
FNM_Q27_13
FNM_Q27_14
FNM_Q27_15
FNM_Q27_16
FNM_Q27_17
FNM_Q27_18
FNM_Q27_19
FNM_Q27_955
FNM_Q27_933
FNM_Q27_open1
FNM_Q28
FNM_Q29
FNM_Q30_m_1
FNM_Q30_m_2
FNM_Q30_m_3
FNM_Q30_m_4
FNM_Q30_m_5
FNM_Q30_m_6
FNM_Q30_m_7
FNM_Q30_m_8
FNM_Q30_m_955
FNM_Q30_m_933
FNM_Q30_open1
FNM_Q31_d
FNM_Q32_1
FNM_Q32_2
FNM_Q32_3
FNM_Q32_4
FNM_Q32_5
FNM_Q32_6
FNM_Q32_7
FNM_Q32_8
FNM_Q32_9
FNM_Q32_10
FNM_Q32_955
FNM_Q32_933
FNM_Q32_open1
FNM_Q33
FNM_34
FNM_Q35_m_1
FNM_Q35_m_2
FNM_Q35_m_3
FNM_Q35_m_933
FNM_Q35_open1
FNM_Q36_d
FNM_Q37_m_1
FNM_Q37_m_2
FNM_Q37_m_3
FNM_Q37_m_4
FNM_Q37_m_5
FNM_Q37_m_6
FNM_Q37_m_7
FNM_Q37_m_8
FNM_Q37_m_9
FNM_Q37_m_955
FNM_Q37_m_933
FNM_Q37_open1
FNM_Q38
F

In [476]:
#FNM_Q24
#FNM_Q29
#FNM_34
#dat.FNM_Q24.unique()
np.unique(dat.FNM_Q24,return_counts = True)

(array([  1,   2, 933]), array([273,  35,   2]))

In [477]:
np.unique(dat.FNM_Q24_recontact,return_counts = True)

(array([ -1,   1,   2, 933]), array([ 51, 229,  28,   2]))

In [478]:
np.unique(dat.FNM_Q29,return_counts = True)

(array([1, 2, 3]), array([269,  38,   3]))

In [479]:
np.unique(dat.FNM_Q29_recontact,return_counts = True)

(array([-1,  1,  2,  3]), array([ 51, 230,  26,   3]))

In [480]:
np.unique(dat.FNM_34,return_counts = True)

(array([  1,   2, 933]), array([110, 198,   2]))

In [481]:
np.unique(dat.FNM_34_recontact,return_counts = True)

(array([ -1,   1,   2, 933]), array([ 51,  90, 166,   3]))

In [482]:
#FNM_Q28
#FNM_Q33
#FNM_Q38


In [483]:
np.unique(dat.FNM_Q28,return_counts = True)

(array([  1,   2, 933, 999]), array([238,  30,   5,  37]))

In [484]:
np.unique(dat.FNM_Q28_recontact,return_counts = True)

(array([ -1,   1,   2, 933, 999]), array([ 51, 213,  14,   2,  30]))

In [485]:
np.unique(dat.FNM_Q33,return_counts = True)

(array([  1,   2, 933, 999]), array([229,  36,   4,  41]))

In [486]:
np.unique(dat.FNM_Q33_recontact,return_counts = True)

(array([ -1,   1,   2, 933, 999]), array([ 51, 214,  15,   1,  29]))

In [487]:
np.unique(dat.FNM_Q38,return_counts = True)

(array([  1,   2, 933, 999]), array([ 89,  19,   2, 200]))

In [488]:
np.unique(dat.FNM_Q38_recontact,return_counts = True)

(array([ -1,   1,   2, 933, 999]), array([ 51,  84,   5,   1, 169]))

In [489]:
# drop the .0 that pandas appends for some reason
dat['caseid'] = dat.caseid.str[:-2]
# i'm not sure what this does
dat = dat.rename(ename_lut, axis=1)

In [490]:
dat

,caseid,weight,sample_type,screener_1ongoing,screener_2impact,screener_3depression,screener_4anxiety,screener_5attention,consent,mood_yn,FNM_Q25_1,FNM_Q25_2,FNM_Q25_3,FNM_Q25_4,FNM_Q25_5,FNM_Q25_6,FNM_Q25_955,FNM_Q25_933,FNM_Q25_open1,mood_years,FNM_Q27_1,FNM_Q27_2,FNM_Q27_3,FNM_Q27_4,FNM_Q27_5,FNM_Q27_6,FNM_Q27_7,FNM_Q27_8,FNM_Q27_9,FNM_Q27_10,FNM_Q27_11,FNM_Q27_12,FNM_Q27_13,FNM_Q27_14,FNM_Q27_15,FNM_Q27_16,FNM_Q27_17,FNM_Q27_18,FNM_Q27_19,FNM_Q27_955,FNM_Q27_933,FNM_Q27_open1,mood_bothered,anxiety_yn,FNM_Q30_m_1,FNM_Q30_m_2,FNM_Q30_m_3,FNM_Q30_m_4,FNM_Q30_m_5,FNM_Q30_m_6,FNM_Q30_m_7,FNM_Q30_m_8,FNM_Q30_m_955,FNM_Q30_m_933,FNM_Q30_open1,anxiety_years,FNM_Q32_1,FNM_Q32_2,FNM_Q32_3,FNM_Q32_4,FNM_Q32_5,FNM_Q32_6,FNM_Q32_7,FNM_Q32_8,FNM_Q32_9,FNM_Q32_10,FNM_Q32_955,FNM_Q32_933,FNM_Q32_open1,anxiety_bothered,attention_yn,FNM_Q35_m_1,FNM_Q35_m_2,FNM_Q35_m_3,FNM_Q35_m_933,FNM_Q35_open1,attention_years,FNM_Q37_m_1,FNM_Q37_m_2,FNM_Q37_m_3,FNM_Q37_m_4,FNM_Q37_m_5,FNM_Q37_m_6,FNM_Q37_m_7,FNM_Q37_m_8,FNM_Q37_m_9,FNM_Q37_m_955,FNM_Q37_m_933,FNM_Q37_open1,attention_bothered,inattention_1,inattention_2,inattention_3,inattention_4,inattention_5,inattention_6,FNM_Q1_attn,inattention_7,inattention_8,inattention_9,hyperactivity_1,hyperactivity_2,hyperactivity_3,hyperactivity_4,hyperactivity_5,impulsivity_1,impulsivity_2,impulsivity_3,impulsivity_4,sct_1,sct_2,sct_3,sct_4,sct_5,sct_6,sct_7,sct_8,sct_9,gad_1,gad_2,gad_3,gad_4,gad_5,gad_6,gad_7,phq_1,phq_2,phq_3,phq_4,phq_5,phq_6,phq_7,phq_8,hitop157,hitop81,hitop34,hitop54,hitop243,hitop182,hitop69,hitop89,hitop50,check_moderately,hitop129,hitop265,hitop124,hitop231,hitop93,hitop67,hitop245,hitop281,hitop141,hitop40,hitop204,hitop21,hitop236,hitop280,hitop84,hitop120,hitop77,hitop92,hitop258,hitop39,hitop254,hitop215,hitop95,hitop106,hitop283,hitop16,hitop20,hitop189,hitop1,hitop136,hitop246,hitop248,hitop257,hitop114,hitop117,hitop250,hitop200,hitop160,hitop23,hitop165,hitop244,hitop9,hitop142,hitop230,hitop149,hitop247,hitop99,hitop66,hitop240,hitop222,hitop90,hitop113,hitop278,hitop203,hitop159,hitop123,hitop275,hitop268,hitop225,hitop143,hitop151,hitop181,hitop211,hitop17,hitop126,hitop5,hitop261,hitop220,check_notatall,hitop15,hitop72,hitop140,hitop109,hitop197,hitop104,todayinattention_1,todayinattention_2,todayinattention_3,todayinattention_4,todayinattention_5,todayinattention_6,todayinattention_7,todayinattention_8,todayinattention_9,todayhyperactivity_1,todayhyperactivity_2,todayhyperactivity_3,todayhyperactivity_4,todayhyperactivity_5,todayimpulsivity_1,todayimpulsivity_2,todayimpulsivity_3,todayimpulsivity_4,todaysct_1,todaysct_2,todaysct_3,todaysct_4,todaysct_5,todaysct_6,todaysct_7,todaysct_8,todaysct_9,today_na1,todaygad_1,todaygad_2,todaygad_3,todaygad_4,...,impulsivity_1_recontact,impulsivity_2_recontact,impulsivity_3_recontact,impulsivity_4_recontact,sct_1_recontact,sct_2_recontact,sct_3_recontact,sct_4_recontact,sct_5_recontact,sct_6_recontact,sct_7_recontact,sct_8_recontact,sct_9_recontact,gad_1_recontact,gad_2_recontact,gad_3_recontact,gad_4_recontact,gad_5_recontact,gad_6_recontact,gad_7_recontact,phq_1_recontact,phq_2_recontact,phq_3_recontact,phq_4_recontact,phq_5_recontact,phq_6_recontact,phq_7_recontact,phq_8_recontact,hitop157_recontact,hitop81_recontact,hitop34_recontact,hitop54_recontact,hitop243_recontact,hitop182_recontact,hitop69_recontact,hitop89_recontact,hitop50_recontact,check_moderately_recontact,hitop129_recontact,hitop265_recontact,hitop124_recontact,hitop231_recontact,hitop93_recontact,hitop67_recontact,hitop245_recontact,hitop281_recontact,hitop141_recontact,hitop40_recontact,hitop204_recontact,hitop21_recontact,hitop236_recontact,hitop280_recontact,hitop84_recontact,hitop120_recontact,hitop77_recontact,hitop92_recontact,hitop258_recontact,hitop39_recontact,hitop254_recontact,hitop215_recontact,hitop95_recontact,hitop106_recontact,hitop283_recontact,hitop16_recontact,hitop20_recontact,hitop189_recontact,hitop1_recontact,hitop136_recontact,hitop246_reconta

In [491]:
for col in dat.columns:
    print(col)

caseid
weight
sample_type
screener_1ongoing
screener_2impact
screener_3depression
screener_4anxiety
screener_5attention
consent
mood_yn
FNM_Q25_1
FNM_Q25_2
FNM_Q25_3
FNM_Q25_4
FNM_Q25_5
FNM_Q25_6
FNM_Q25_955
FNM_Q25_933
FNM_Q25_open1
mood_years
FNM_Q27_1
FNM_Q27_2
FNM_Q27_3
FNM_Q27_4
FNM_Q27_5
FNM_Q27_6
FNM_Q27_7
FNM_Q27_8
FNM_Q27_9
FNM_Q27_10
FNM_Q27_11
FNM_Q27_12
FNM_Q27_13
FNM_Q27_14
FNM_Q27_15
FNM_Q27_16
FNM_Q27_17
FNM_Q27_18
FNM_Q27_19
FNM_Q27_955
FNM_Q27_933
FNM_Q27_open1
mood_bothered
anxiety_yn
FNM_Q30_m_1
FNM_Q30_m_2
FNM_Q30_m_3
FNM_Q30_m_4
FNM_Q30_m_5
FNM_Q30_m_6
FNM_Q30_m_7
FNM_Q30_m_8
FNM_Q30_m_955
FNM_Q30_m_933
FNM_Q30_open1
anxiety_years
FNM_Q32_1
FNM_Q32_2
FNM_Q32_3
FNM_Q32_4
FNM_Q32_5
FNM_Q32_6
FNM_Q32_7
FNM_Q32_8
FNM_Q32_9
FNM_Q32_10
FNM_Q32_955
FNM_Q32_933
FNM_Q32_open1
anxiety_bothered
attention_yn
FNM_Q35_m_1
FNM_Q35_m_2
FNM_Q35_m_3
FNM_Q35_m_933
FNM_Q35_open1
attention_years
FNM_Q37_m_1
FNM_Q37_m_2
FNM_Q37_m_3
FNM_Q37_m_4
FNM_Q37_m_5
FNM_Q37_m_6
FNM_Q37_m_7
FNM_Q37

In [492]:
# renaming columns
dat = dat.rename(columns={
    'mood_bothered': 'mood_bothered_orig',
    'mood_bothered_recontact': 'mood_bothered_recontact_orig',
    'anxiety_bothered': 'anxiety_bothered_orig',
    'anxiety_bothered_recontact': 'anxiety_bothered_recontact_orig',
    'attention_bothered': 'attention_bothered_orig',
    'attention_bothered_recontact': 'attention_bothered_recontact_orig'})

In [493]:
# clean mood diagnosis
dat['mood_dx'] = ''
dat['n_mood_dx'] = (dat.loc[:, ['FNM_Q25_1', 'FNM_Q25_2', 'FNM_Q25_3', 'FNM_Q25_4', 'FNM_Q25_5', 'FNM_Q25_6']]==1).sum(1)
dat.loc[dat.FNM_Q25_955 == 1, 'mood_dx'] = 'other'
dat.loc[dat.FNM_Q25_1 == 1, 'mood_dx'] = 'mdd'
dat.loc[dat.FNM_Q25_2 == 1, 'mood_dx'] = 'persistent'
dat.loc[dat.FNM_Q25_3 == 1, 'mood_dx'] = 'premenstrual'
dat.loc[dat.FNM_Q25_4 == 1, 'mood_dx'] = 'bipolarI'
dat.loc[dat.FNM_Q25_5 == 1, 'mood_dx'] = 'bipolarII'
dat.loc[dat.FNM_Q25_6 == 1, 'mood_dx'] = 'cyclothymic'
dat.loc[dat.n_mood_dx > 1, 'mood_dx'] = 'multiple'

# clean other mood columns
dat.loc[dat.mood_years == 999, 'mood_years'] = np.nan

dat['mood_bothered']=False
dat.loc[dat.mood_bothered_orig == 1, 'mood_bothered'] = True

dat

,caseid,weight,sample_type,screener_1ongoing,screener_2impact,screener_3depression,screener_4anxiety,screener_5attention,consent,mood_yn,FNM_Q25_1,FNM_Q25_2,FNM_Q25_3,FNM_Q25_4,FNM_Q25_5,FNM_Q25_6,FNM_Q25_955,FNM_Q25_933,FNM_Q25_open1,mood_years,FNM_Q27_1,FNM_Q27_2,FNM_Q27_3,FNM_Q27_4,FNM_Q27_5,FNM_Q27_6,FNM_Q27_7,FNM_Q27_8,FNM_Q27_9,FNM_Q27_10,FNM_Q27_11,FNM_Q27_12,FNM_Q27_13,FNM_Q27_14,FNM_Q27_15,FNM_Q27_16,FNM_Q27_17,FNM_Q27_18,FNM_Q27_19,FNM_Q27_955,FNM_Q27_933,FNM_Q27_open1,mood_bothered_orig,anxiety_yn,FNM_Q30_m_1,FNM_Q30_m_2,FNM_Q30_m_3,FNM_Q30_m_4,FNM_Q30_m_5,FNM_Q30_m_6,FNM_Q30_m_7,FNM_Q30_m_8,FNM_Q30_m_955,FNM_Q30_m_933,FNM_Q30_open1,anxiety_years,FNM_Q32_1,FNM_Q32_2,FNM_Q32_3,FNM_Q32_4,FNM_Q32_5,FNM_Q32_6,FNM_Q32_7,FNM_Q32_8,FNM_Q32_9,FNM_Q32_10,FNM_Q32_955,FNM_Q32_933,FNM_Q32_open1,anxiety_bothered_orig,attention_yn,FNM_Q35_m_1,FNM_Q35_m_2,FNM_Q35_m_3,FNM_Q35_m_933,FNM_Q35_open1,attention_years,FNM_Q37_m_1,FNM_Q37_m_2,FNM_Q37_m_3,FNM_Q37_m_4,FNM_Q37_m_5,FNM_Q37_m_6,FNM_Q37_m_7,FNM_Q37_m_8,FNM_Q37_m_9,FNM_Q37_m_955,FNM_Q37_m_933,FNM_Q37_open1,attention_bothered_orig,inattention_1,inattention_2,inattention_3,inattention_4,inattention_5,inattention_6,FNM_Q1_attn,inattention_7,inattention_8,inattention_9,hyperactivity_1,hyperactivity_2,hyperactivity_3,hyperactivity_4,hyperactivity_5,impulsivity_1,impulsivity_2,impulsivity_3,impulsivity_4,sct_1,sct_2,sct_3,sct_4,sct_5,sct_6,sct_7,sct_8,sct_9,gad_1,gad_2,gad_3,gad_4,gad_5,gad_6,gad_7,phq_1,phq_2,phq_3,phq_4,phq_5,phq_6,phq_7,phq_8,hitop157,hitop81,hitop34,hitop54,hitop243,hitop182,hitop69,hitop89,hitop50,check_moderately,hitop129,hitop265,hitop124,hitop231,hitop93,hitop67,hitop245,hitop281,hitop141,hitop40,hitop204,hitop21,hitop236,hitop280,hitop84,hitop120,hitop77,hitop92,hitop258,hitop39,hitop254,hitop215,hitop95,hitop106,hitop283,hitop16,hitop20,hitop189,hitop1,hitop136,hitop246,hitop248,hitop257,hitop114,hitop117,hitop250,hitop200,hitop160,hitop23,hitop165,hitop244,hitop9,hitop142,hitop230,hitop149,hitop247,hitop99,hitop66,hitop240,hitop222,hitop90,hitop113,hitop278,hitop203,hitop159,hitop123,hitop275,hitop268,hitop225,hitop143,hitop151,hitop181,hitop211,hitop17,hitop126,hitop5,hitop261,hitop220,check_notatall,hitop15,hitop72,hitop140,hitop109,hitop197,hitop104,todayinattention_1,todayinattention_2,todayinattention_3,todayinattention_4,todayinattention_5,todayinattention_6,todayinattention_7,todayinattention_8,todayinattention_9,todayhyperactivity_1,todayhyperactivity_2,todayhyperactivity_3,todayhyperactivity_4,todayhyperactivity_5,todayimpulsivity_1,todayimpulsivity_2,todayimpulsivity_3,todayimpulsivity_4,todaysct_1,todaysct_2,todaysct_3,todaysct_4,todaysct_5,todaysct_6,todaysct_7,todaysct_8,todaysct_9,today_na1,todaygad_1,todaygad_2,todaygad_3,todaygad_4,...,impulsivity_4_recontact,sct_1_recontact,sct_2_recontact,sct_3_recontact,sct_4_recontact,sct_5_recontact,sct_6_recontact,sct_7_recontact,sct_8_recontact,sct_9_recontact,gad_1_recontact,gad_2_recontact,gad_3_recontact,gad_4_recontact,gad_5_recontact,gad_6_recontact,gad_7_recontact,phq_1_recontact,phq_2_recontact,phq_3_recontact,phq_4_recontact,phq_5_recontact,phq_6_recontact,phq_7_recontact,phq_8_recontact,hitop157_recontact,hitop81_recontact,hitop34_recontact,hitop54_recontact,hitop243_recontact,hitop182_recontact,hitop69_recontact,hitop89_recontact,hitop50_recontact,check_moderately_recontact,hitop129_recontact,hitop265_recontact,hitop124_recontact,hitop231_recontact,hitop93_recontact,hitop67_recontact,hitop245_recontact,hitop281_recontact,hitop141_recontact,hitop40_recontact,hitop204_recontact,hitop21_recontact,hitop236_recontact,hitop280_recontact,hitop84_recontact,hitop120_recontact,hitop77_recontact,hitop92_recontact,hitop258_recontact,hitop39_recontact,hitop254_recontact,hitop215_recontact,hitop95_recontact,hitop106_recontact,hitop283_recontact,hitop16_recontact,hitop20_recontact,hitop189_recontact,hitop1_recontact,hitop136_recontact,hitop246_recontact,hitop248_recontact,hitop257_recontact,hitop114_reconta

In [494]:
# clean mood diagnosis - recontact
dat['mood_dx_recontact'] = ''
dat['n_mood_dx_recontact'] = (dat.loc[:, ['FNM_Q25_1_recontact', 'FNM_Q25_2_recontact', 'FNM_Q25_3_recontact', 'FNM_Q25_4_recontact', 'FNM_Q25_5_recontact', 'FNM_Q25_6_recontact']]==1).sum(1)
dat.loc[dat.FNM_Q25_955_recontact == 1, 'mood_dx_recontact'] = 'other'
dat.loc[dat.FNM_Q25_1_recontact == 1, 'mood_dx_recontact'] = 'mdd'
dat.loc[dat.FNM_Q25_2_recontact == 1, 'mood_dx_recontact'] = 'persistent'
dat.loc[dat.FNM_Q25_3_recontact == 1, 'mood_dx_recontact'] = 'premenstrual'
dat.loc[dat.FNM_Q25_4_recontact == 1, 'mood_dx_recontact'] = 'bipolarI'
dat.loc[dat.FNM_Q25_5_recontact == 1, 'mood_dx_recontact'] = 'bipolarII'
dat.loc[dat.FNM_Q25_6_recontact == 1, 'mood_dx_recontact'] = 'cyclothymic'
dat.loc[dat.n_mood_dx_recontact > 1, 'mood_dx_recontact'] = 'multiple'

# clean other mood columns - recontact
# clean other mood columns
dat.loc[dat.mood_years_recontact == 999, 'mood_years_recontact'] = np.nan

dat['mood_bothered_recontact']=False
dat.loc[dat.mood_bothered_recontact_orig == 1, 'mood_bothered_recontact'] = True

dat

,caseid,weight,sample_type,screener_1ongoing,screener_2impact,screener_3depression,screener_4anxiety,screener_5attention,consent,mood_yn,FNM_Q25_1,FNM_Q25_2,FNM_Q25_3,FNM_Q25_4,FNM_Q25_5,FNM_Q25_6,FNM_Q25_955,FNM_Q25_933,FNM_Q25_open1,mood_years,FNM_Q27_1,FNM_Q27_2,FNM_Q27_3,FNM_Q27_4,FNM_Q27_5,FNM_Q27_6,FNM_Q27_7,FNM_Q27_8,FNM_Q27_9,FNM_Q27_10,FNM_Q27_11,FNM_Q27_12,FNM_Q27_13,FNM_Q27_14,FNM_Q27_15,FNM_Q27_16,FNM_Q27_17,FNM_Q27_18,FNM_Q27_19,FNM_Q27_955,FNM_Q27_933,FNM_Q27_open1,mood_bothered_orig,anxiety_yn,FNM_Q30_m_1,FNM_Q30_m_2,FNM_Q30_m_3,FNM_Q30_m_4,FNM_Q30_m_5,FNM_Q30_m_6,FNM_Q30_m_7,FNM_Q30_m_8,FNM_Q30_m_955,FNM_Q30_m_933,FNM_Q30_open1,anxiety_years,FNM_Q32_1,FNM_Q32_2,FNM_Q32_3,FNM_Q32_4,FNM_Q32_5,FNM_Q32_6,FNM_Q32_7,FNM_Q32_8,FNM_Q32_9,FNM_Q32_10,FNM_Q32_955,FNM_Q32_933,FNM_Q32_open1,anxiety_bothered_orig,attention_yn,FNM_Q35_m_1,FNM_Q35_m_2,FNM_Q35_m_3,FNM_Q35_m_933,FNM_Q35_open1,attention_years,FNM_Q37_m_1,FNM_Q37_m_2,FNM_Q37_m_3,FNM_Q37_m_4,FNM_Q37_m_5,FNM_Q37_m_6,FNM_Q37_m_7,FNM_Q37_m_8,FNM_Q37_m_9,FNM_Q37_m_955,FNM_Q37_m_933,FNM_Q37_open1,attention_bothered_orig,inattention_1,inattention_2,inattention_3,inattention_4,inattention_5,inattention_6,FNM_Q1_attn,inattention_7,inattention_8,inattention_9,hyperactivity_1,hyperactivity_2,hyperactivity_3,hyperactivity_4,hyperactivity_5,impulsivity_1,impulsivity_2,impulsivity_3,impulsivity_4,sct_1,sct_2,sct_3,sct_4,sct_5,sct_6,sct_7,sct_8,sct_9,gad_1,gad_2,gad_3,gad_4,gad_5,gad_6,gad_7,phq_1,phq_2,phq_3,phq_4,phq_5,phq_6,phq_7,phq_8,hitop157,hitop81,hitop34,hitop54,hitop243,hitop182,hitop69,hitop89,hitop50,check_moderately,hitop129,hitop265,hitop124,hitop231,hitop93,hitop67,hitop245,hitop281,hitop141,hitop40,hitop204,hitop21,hitop236,hitop280,hitop84,hitop120,hitop77,hitop92,hitop258,hitop39,hitop254,hitop215,hitop95,hitop106,hitop283,hitop16,hitop20,hitop189,hitop1,hitop136,hitop246,hitop248,hitop257,hitop114,hitop117,hitop250,hitop200,hitop160,hitop23,hitop165,hitop244,hitop9,hitop142,hitop230,hitop149,hitop247,hitop99,hitop66,hitop240,hitop222,hitop90,hitop113,hitop278,hitop203,hitop159,hitop123,hitop275,hitop268,hitop225,hitop143,hitop151,hitop181,hitop211,hitop17,hitop126,hitop5,hitop261,hitop220,check_notatall,hitop15,hitop72,hitop140,hitop109,hitop197,hitop104,todayinattention_1,todayinattention_2,todayinattention_3,todayinattention_4,todayinattention_5,todayinattention_6,todayinattention_7,todayinattention_8,todayinattention_9,todayhyperactivity_1,todayhyperactivity_2,todayhyperactivity_3,todayhyperactivity_4,todayhyperactivity_5,todayimpulsivity_1,todayimpulsivity_2,todayimpulsivity_3,todayimpulsivity_4,todaysct_1,todaysct_2,todaysct_3,todaysct_4,todaysct_5,todaysct_6,todaysct_7,todaysct_8,todaysct_9,today_na1,todaygad_1,todaygad_2,todaygad_3,todaygad_4,...,sct_3_recontact,sct_4_recontact,sct_5_recontact,sct_6_recontact,sct_7_recontact,sct_8_recontact,sct_9_recontact,gad_1_recontact,gad_2_recontact,gad_3_recontact,gad_4_recontact,gad_5_recontact,gad_6_recontact,gad_7_recontact,phq_1_recontact,phq_2_recontact,phq_3_recontact,phq_4_recontact,phq_5_recontact,phq_6_recontact,phq_7_recontact,phq_8_recontact,hitop157_recontact,hitop81_recontact,hitop34_recontact,hitop54_recontact,hitop243_recontact,hitop182_recontact,hitop69_recontact,hitop89_recontact,hitop50_recontact,check_moderately_recontact,hitop129_recontact,hitop265_recontact,hitop124_recontact,hitop231_recontact,hitop93_recontact,hitop67_recontact,hitop245_recontact,hitop281_recontact,hitop141_recontact,hitop40_recontact,hitop204_recontact,hitop21_recontact,hitop236_recontact,hitop280_recontact,hitop84_recontact,hitop120_recontact,hitop77_recontact,hitop92_recontact,hitop258_recontact,hitop39_recontact,hitop254_recontact,hitop215_recontact,hitop95_recontact,hitop106_recontact,hitop283_recontact,hitop16_recontact,hitop20_recontact,hitop189_recontact,hitop1_recontact,hitop136_recontact,hitop246_recontact,hitop248_recontact,hitop257_recontact,hitop114_recontact,hitop117_recontact,hitop250_recontact,hitop200_recont

In [495]:
# clean anxiety diagnosis
dat['anxiety_dx'] = ''
dat['n_anxiety_dx'] = (dat.loc[:, ['FNM_Q30_m_1',
                                   'FNM_Q30_m_2',
                                   'FNM_Q30_m_3', 
                                   'FNM_Q30_m_4', 
                                   'FNM_Q30_m_5',
                                   'FNM_Q30_m_6',
                                   'FNM_Q30_m_7',
                                   'FNM_Q30_m_8',]]==1).sum(1)
dat.loc[dat.FNM_Q30_m_955 == 1, 'anxiety_dx'] = 'other'
dat.loc[dat.FNM_Q30_m_1 == 1, 'anxiety_dx'] = 'gad'
dat.loc[dat.FNM_Q30_m_2 == 1, 'anxiety_dx'] = 'separation'
dat.loc[dat.FNM_Q30_m_3 == 1, 'anxiety_dx'] = 'agoraphobia'
dat.loc[dat.FNM_Q30_m_4 == 1, 'anxiety_dx'] = 'phobia'
dat.loc[dat.FNM_Q30_m_5 == 1, 'anxiety_dx'] = 'social'
dat.loc[dat.FNM_Q30_m_6 == 1, 'anxiety_dx'] = 'panic_disorder'
dat.loc[dat.FNM_Q30_m_7 == 1, 'anxiety_dx'] = 'panic_attack'
dat.loc[dat.FNM_Q30_m_8 == 1, 'anxiety_dx'] = 'mutism'

dat.loc[dat.n_anxiety_dx > 1, 'anxiety_dx'] = 'multiple'

# clean other anxiety columns

dat.loc[dat.anxiety_years == 999, 'anxiety_years'] = np.nan

dat['anxiety_bothered']=False
dat.loc[dat.anxiety_bothered_orig == 1, 'anxiety_bothered'] = True

dat

,caseid,weight,sample_type,screener_1ongoing,screener_2impact,screener_3depression,screener_4anxiety,screener_5attention,consent,mood_yn,FNM_Q25_1,FNM_Q25_2,FNM_Q25_3,FNM_Q25_4,FNM_Q25_5,FNM_Q25_6,FNM_Q25_955,FNM_Q25_933,FNM_Q25_open1,mood_years,FNM_Q27_1,FNM_Q27_2,FNM_Q27_3,FNM_Q27_4,FNM_Q27_5,FNM_Q27_6,FNM_Q27_7,FNM_Q27_8,FNM_Q27_9,FNM_Q27_10,FNM_Q27_11,FNM_Q27_12,FNM_Q27_13,FNM_Q27_14,FNM_Q27_15,FNM_Q27_16,FNM_Q27_17,FNM_Q27_18,FNM_Q27_19,FNM_Q27_955,FNM_Q27_933,FNM_Q27_open1,mood_bothered_orig,anxiety_yn,FNM_Q30_m_1,FNM_Q30_m_2,FNM_Q30_m_3,FNM_Q30_m_4,FNM_Q30_m_5,FNM_Q30_m_6,FNM_Q30_m_7,FNM_Q30_m_8,FNM_Q30_m_955,FNM_Q30_m_933,FNM_Q30_open1,anxiety_years,FNM_Q32_1,FNM_Q32_2,FNM_Q32_3,FNM_Q32_4,FNM_Q32_5,FNM_Q32_6,FNM_Q32_7,FNM_Q32_8,FNM_Q32_9,FNM_Q32_10,FNM_Q32_955,FNM_Q32_933,FNM_Q32_open1,anxiety_bothered_orig,attention_yn,FNM_Q35_m_1,FNM_Q35_m_2,FNM_Q35_m_3,FNM_Q35_m_933,FNM_Q35_open1,attention_years,FNM_Q37_m_1,FNM_Q37_m_2,FNM_Q37_m_3,FNM_Q37_m_4,FNM_Q37_m_5,FNM_Q37_m_6,FNM_Q37_m_7,FNM_Q37_m_8,FNM_Q37_m_9,FNM_Q37_m_955,FNM_Q37_m_933,FNM_Q37_open1,attention_bothered_orig,inattention_1,inattention_2,inattention_3,inattention_4,inattention_5,inattention_6,FNM_Q1_attn,inattention_7,inattention_8,inattention_9,hyperactivity_1,hyperactivity_2,hyperactivity_3,hyperactivity_4,hyperactivity_5,impulsivity_1,impulsivity_2,impulsivity_3,impulsivity_4,sct_1,sct_2,sct_3,sct_4,sct_5,sct_6,sct_7,sct_8,sct_9,gad_1,gad_2,gad_3,gad_4,gad_5,gad_6,gad_7,phq_1,phq_2,phq_3,phq_4,phq_5,phq_6,phq_7,phq_8,hitop157,hitop81,hitop34,hitop54,hitop243,hitop182,hitop69,hitop89,hitop50,check_moderately,hitop129,hitop265,hitop124,hitop231,hitop93,hitop67,hitop245,hitop281,hitop141,hitop40,hitop204,hitop21,hitop236,hitop280,hitop84,hitop120,hitop77,hitop92,hitop258,hitop39,hitop254,hitop215,hitop95,hitop106,hitop283,hitop16,hitop20,hitop189,hitop1,hitop136,hitop246,hitop248,hitop257,hitop114,hitop117,hitop250,hitop200,hitop160,hitop23,hitop165,hitop244,hitop9,hitop142,hitop230,hitop149,hitop247,hitop99,hitop66,hitop240,hitop222,hitop90,hitop113,hitop278,hitop203,hitop159,hitop123,hitop275,hitop268,hitop225,hitop143,hitop151,hitop181,hitop211,hitop17,hitop126,hitop5,hitop261,hitop220,check_notatall,hitop15,hitop72,hitop140,hitop109,hitop197,hitop104,todayinattention_1,todayinattention_2,todayinattention_3,todayinattention_4,todayinattention_5,todayinattention_6,todayinattention_7,todayinattention_8,todayinattention_9,todayhyperactivity_1,todayhyperactivity_2,todayhyperactivity_3,todayhyperactivity_4,todayhyperactivity_5,todayimpulsivity_1,todayimpulsivity_2,todayimpulsivity_3,todayimpulsivity_4,todaysct_1,todaysct_2,todaysct_3,todaysct_4,todaysct_5,todaysct_6,todaysct_7,todaysct_8,todaysct_9,today_na1,todaygad_1,todaygad_2,todaygad_3,todaygad_4,...,sct_6_recontact,sct_7_recontact,sct_8_recontact,sct_9_recontact,gad_1_recontact,gad_2_recontact,gad_3_recontact,gad_4_recontact,gad_5_recontact,gad_6_recontact,gad_7_recontact,phq_1_recontact,phq_2_recontact,phq_3_recontact,phq_4_recontact,phq_5_recontact,phq_6_recontact,phq_7_recontact,phq_8_recontact,hitop157_recontact,hitop81_recontact,hitop34_recontact,hitop54_recontact,hitop243_recontact,hitop182_recontact,hitop69_recontact,hitop89_recontact,hitop50_recontact,check_moderately_recontact,hitop129_recontact,hitop265_recontact,hitop124_recontact,hitop231_recontact,hitop93_recontact,hitop67_recontact,hitop245_recontact,hitop281_recontact,hitop141_recontact,hitop40_recontact,hitop204_recontact,hitop21_recontact,hitop236_recontact,hitop280_recontact,hitop84_recontact,hitop120_recontact,hitop77_recontact,hitop92_recontact,hitop258_recontact,hitop39_recontact,hitop254_recontact,hitop215_recontact,hitop95_recontact,hitop106_recontact,hitop283_recontact,hitop16_recontact,hitop20_recontact,hitop189_recontact,hitop1_recontact,hitop136_recontact,hitop246_recontact,hitop248_recontact,hitop257_recontact,hitop114_recontact,hitop117_recontact,hitop250_recontact,hitop200_recontact,hitop160_recontact,hitop23_recontact,hitop16

In [496]:
# clean anxiety diagnosis - recontact
dat['anxiety_dx_recontact'] = ''
dat['n_anxiety_dx_recontact'] = (dat.loc[:, ['FNM_Q30_m_1_recontact',
                                   'FNM_Q30_m_2_recontact',
                                   'FNM_Q30_m_3_recontact', 
                                   'FNM_Q30_m_4_recontact', 
                                   'FNM_Q30_m_5_recontact',
                                   'FNM_Q30_m_6_recontact',
                                   'FNM_Q30_m_7_recontact',
                                   'FNM_Q30_m_8_recontact',]]==1).sum(1)
dat.loc[dat.FNM_Q30_m_955_recontact == 1, 'anxiety_dx_recontact'] = 'other'
dat.loc[dat.FNM_Q30_m_1_recontact == 1, 'anxiety_dx_recontact'] = 'gad'
dat.loc[dat.FNM_Q30_m_2_recontact == 1, 'anxiety_dx_recontact'] = 'separation'
dat.loc[dat.FNM_Q30_m_3_recontact == 1, 'anxiety_dx_recontact'] = 'agoraphobia'
dat.loc[dat.FNM_Q30_m_4_recontact == 1, 'anxiety_dx_recontact'] = 'phobia'
dat.loc[dat.FNM_Q30_m_5_recontact == 1, 'anxiety_dx_recontact'] = 'social'
dat.loc[dat.FNM_Q30_m_6_recontact == 1, 'anxiety_dx_recontact'] = 'panic_disorder'
dat.loc[dat.FNM_Q30_m_7_recontact == 1, 'anxiety_dx_recontact'] = 'panic_attack'
dat.loc[dat.FNM_Q30_m_8_recontact == 1, 'anxiety_dx_recontact'] = 'mutism'

dat.loc[dat.n_anxiety_dx_recontact > 1, 'anxiety_dx_recontact'] = 'multiple'

# clean other anxiety columns - recontact
dat.loc[dat.anxiety_years_recontact == 999, 'anxiety_years_recontact'] = np.nan
dat['anxiety_bothered_recontact']=False
dat.loc[dat.anxiety_bothered_recontact_orig == 1, 'anxiety_bothered_recontact'] = True

dat

,caseid,weight,sample_type,screener_1ongoing,screener_2impact,screener_3depression,screener_4anxiety,screener_5attention,consent,mood_yn,FNM_Q25_1,FNM_Q25_2,FNM_Q25_3,FNM_Q25_4,FNM_Q25_5,FNM_Q25_6,FNM_Q25_955,FNM_Q25_933,FNM_Q25_open1,mood_years,FNM_Q27_1,FNM_Q27_2,FNM_Q27_3,FNM_Q27_4,FNM_Q27_5,FNM_Q27_6,FNM_Q27_7,FNM_Q27_8,FNM_Q27_9,FNM_Q27_10,FNM_Q27_11,FNM_Q27_12,FNM_Q27_13,FNM_Q27_14,FNM_Q27_15,FNM_Q27_16,FNM_Q27_17,FNM_Q27_18,FNM_Q27_19,FNM_Q27_955,FNM_Q27_933,FNM_Q27_open1,mood_bothered_orig,anxiety_yn,FNM_Q30_m_1,FNM_Q30_m_2,FNM_Q30_m_3,FNM_Q30_m_4,FNM_Q30_m_5,FNM_Q30_m_6,FNM_Q30_m_7,FNM_Q30_m_8,FNM_Q30_m_955,FNM_Q30_m_933,FNM_Q30_open1,anxiety_years,FNM_Q32_1,FNM_Q32_2,FNM_Q32_3,FNM_Q32_4,FNM_Q32_5,FNM_Q32_6,FNM_Q32_7,FNM_Q32_8,FNM_Q32_9,FNM_Q32_10,FNM_Q32_955,FNM_Q32_933,FNM_Q32_open1,anxiety_bothered_orig,attention_yn,FNM_Q35_m_1,FNM_Q35_m_2,FNM_Q35_m_3,FNM_Q35_m_933,FNM_Q35_open1,attention_years,FNM_Q37_m_1,FNM_Q37_m_2,FNM_Q37_m_3,FNM_Q37_m_4,FNM_Q37_m_5,FNM_Q37_m_6,FNM_Q37_m_7,FNM_Q37_m_8,FNM_Q37_m_9,FNM_Q37_m_955,FNM_Q37_m_933,FNM_Q37_open1,attention_bothered_orig,inattention_1,inattention_2,inattention_3,inattention_4,inattention_5,inattention_6,FNM_Q1_attn,inattention_7,inattention_8,inattention_9,hyperactivity_1,hyperactivity_2,hyperactivity_3,hyperactivity_4,hyperactivity_5,impulsivity_1,impulsivity_2,impulsivity_3,impulsivity_4,sct_1,sct_2,sct_3,sct_4,sct_5,sct_6,sct_7,sct_8,sct_9,gad_1,gad_2,gad_3,gad_4,gad_5,gad_6,gad_7,phq_1,phq_2,phq_3,phq_4,phq_5,phq_6,phq_7,phq_8,hitop157,hitop81,hitop34,hitop54,hitop243,hitop182,hitop69,hitop89,hitop50,check_moderately,hitop129,hitop265,hitop124,hitop231,hitop93,hitop67,hitop245,hitop281,hitop141,hitop40,hitop204,hitop21,hitop236,hitop280,hitop84,hitop120,hitop77,hitop92,hitop258,hitop39,hitop254,hitop215,hitop95,hitop106,hitop283,hitop16,hitop20,hitop189,hitop1,hitop136,hitop246,hitop248,hitop257,hitop114,hitop117,hitop250,hitop200,hitop160,hitop23,hitop165,hitop244,hitop9,hitop142,hitop230,hitop149,hitop247,hitop99,hitop66,hitop240,hitop222,hitop90,hitop113,hitop278,hitop203,hitop159,hitop123,hitop275,hitop268,hitop225,hitop143,hitop151,hitop181,hitop211,hitop17,hitop126,hitop5,hitop261,hitop220,check_notatall,hitop15,hitop72,hitop140,hitop109,hitop197,hitop104,todayinattention_1,todayinattention_2,todayinattention_3,todayinattention_4,todayinattention_5,todayinattention_6,todayinattention_7,todayinattention_8,todayinattention_9,todayhyperactivity_1,todayhyperactivity_2,todayhyperactivity_3,todayhyperactivity_4,todayhyperactivity_5,todayimpulsivity_1,todayimpulsivity_2,todayimpulsivity_3,todayimpulsivity_4,todaysct_1,todaysct_2,todaysct_3,todaysct_4,todaysct_5,todaysct_6,todaysct_7,todaysct_8,todaysct_9,today_na1,todaygad_1,todaygad_2,todaygad_3,todaygad_4,...,sct_9_recontact,gad_1_recontact,gad_2_recontact,gad_3_recontact,gad_4_recontact,gad_5_recontact,gad_6_recontact,gad_7_recontact,phq_1_recontact,phq_2_recontact,phq_3_recontact,phq_4_recontact,phq_5_recontact,phq_6_recontact,phq_7_recontact,phq_8_recontact,hitop157_recontact,hitop81_recontact,hitop34_recontact,hitop54_recontact,hitop243_recontact,hitop182_recontact,hitop69_recontact,hitop89_recontact,hitop50_recontact,check_moderately_recontact,hitop129_recontact,hitop265_recontact,hitop124_recontact,hitop231_recontact,hitop93_recontact,hitop67_recontact,hitop245_recontact,hitop281_recontact,hitop141_recontact,hitop40_recontact,hitop204_recontact,hitop21_recontact,hitop236_recontact,hitop280_recontact,hitop84_recontact,hitop120_recontact,hitop77_recontact,hitop92_recontact,hitop258_recontact,hitop39_recontact,hitop254_recontact,hitop215_recontact,hitop95_recontact,hitop106_recontact,hitop283_recontact,hitop16_recontact,hitop20_recontact,hitop189_recontact,hitop1_recontact,hitop136_recontact,hitop246_recontact,hitop248_recontact,hitop257_recontact,hitop114_recontact,hitop117_recontact,hitop250_recontact,hitop200_recontact,hitop160_recontact,hitop23_recontact,hitop165_recontact,hitop244_recontact,hitop9_recontact,

In [497]:
# clean attention diagnosis
dat['attention_dx'] = ''
dat['n_attention_dx'] = (dat.loc[:, ['FNM_Q35_m_1',
                                   'FNM_Q35_m_2',
                                  ]]==1).sum(1)
dat.loc[dat.FNM_Q35_m_3 == 1, 'attention_dx'] = 'other'
dat.loc[dat.FNM_Q35_m_1 == 1, 'attention_dx'] = 'adhd'
dat.loc[dat.FNM_Q35_m_2 == 1, 'attention_dx'] = 'adhd'

# clean other attention columns
dat.loc[dat.attention_years == 999, 'attention_years'] = np.nan
dat['attention_bothered']=False
dat.loc[dat.attention_bothered_orig == 1, 'attention_bothered'] = True

dat

,caseid,weight,sample_type,screener_1ongoing,screener_2impact,screener_3depression,screener_4anxiety,screener_5attention,consent,mood_yn,FNM_Q25_1,FNM_Q25_2,FNM_Q25_3,FNM_Q25_4,FNM_Q25_5,FNM_Q25_6,FNM_Q25_955,FNM_Q25_933,FNM_Q25_open1,mood_years,FNM_Q27_1,FNM_Q27_2,FNM_Q27_3,FNM_Q27_4,FNM_Q27_5,FNM_Q27_6,FNM_Q27_7,FNM_Q27_8,FNM_Q27_9,FNM_Q27_10,FNM_Q27_11,FNM_Q27_12,FNM_Q27_13,FNM_Q27_14,FNM_Q27_15,FNM_Q27_16,FNM_Q27_17,FNM_Q27_18,FNM_Q27_19,FNM_Q27_955,FNM_Q27_933,FNM_Q27_open1,mood_bothered_orig,anxiety_yn,FNM_Q30_m_1,FNM_Q30_m_2,FNM_Q30_m_3,FNM_Q30_m_4,FNM_Q30_m_5,FNM_Q30_m_6,FNM_Q30_m_7,FNM_Q30_m_8,FNM_Q30_m_955,FNM_Q30_m_933,FNM_Q30_open1,anxiety_years,FNM_Q32_1,FNM_Q32_2,FNM_Q32_3,FNM_Q32_4,FNM_Q32_5,FNM_Q32_6,FNM_Q32_7,FNM_Q32_8,FNM_Q32_9,FNM_Q32_10,FNM_Q32_955,FNM_Q32_933,FNM_Q32_open1,anxiety_bothered_orig,attention_yn,FNM_Q35_m_1,FNM_Q35_m_2,FNM_Q35_m_3,FNM_Q35_m_933,FNM_Q35_open1,attention_years,FNM_Q37_m_1,FNM_Q37_m_2,FNM_Q37_m_3,FNM_Q37_m_4,FNM_Q37_m_5,FNM_Q37_m_6,FNM_Q37_m_7,FNM_Q37_m_8,FNM_Q37_m_9,FNM_Q37_m_955,FNM_Q37_m_933,FNM_Q37_open1,attention_bothered_orig,inattention_1,inattention_2,inattention_3,inattention_4,inattention_5,inattention_6,FNM_Q1_attn,inattention_7,inattention_8,inattention_9,hyperactivity_1,hyperactivity_2,hyperactivity_3,hyperactivity_4,hyperactivity_5,impulsivity_1,impulsivity_2,impulsivity_3,impulsivity_4,sct_1,sct_2,sct_3,sct_4,sct_5,sct_6,sct_7,sct_8,sct_9,gad_1,gad_2,gad_3,gad_4,gad_5,gad_6,gad_7,phq_1,phq_2,phq_3,phq_4,phq_5,phq_6,phq_7,phq_8,hitop157,hitop81,hitop34,hitop54,hitop243,hitop182,hitop69,hitop89,hitop50,check_moderately,hitop129,hitop265,hitop124,hitop231,hitop93,hitop67,hitop245,hitop281,hitop141,hitop40,hitop204,hitop21,hitop236,hitop280,hitop84,hitop120,hitop77,hitop92,hitop258,hitop39,hitop254,hitop215,hitop95,hitop106,hitop283,hitop16,hitop20,hitop189,hitop1,hitop136,hitop246,hitop248,hitop257,hitop114,hitop117,hitop250,hitop200,hitop160,hitop23,hitop165,hitop244,hitop9,hitop142,hitop230,hitop149,hitop247,hitop99,hitop66,hitop240,hitop222,hitop90,hitop113,hitop278,hitop203,hitop159,hitop123,hitop275,hitop268,hitop225,hitop143,hitop151,hitop181,hitop211,hitop17,hitop126,hitop5,hitop261,hitop220,check_notatall,hitop15,hitop72,hitop140,hitop109,hitop197,hitop104,todayinattention_1,todayinattention_2,todayinattention_3,todayinattention_4,todayinattention_5,todayinattention_6,todayinattention_7,todayinattention_8,todayinattention_9,todayhyperactivity_1,todayhyperactivity_2,todayhyperactivity_3,todayhyperactivity_4,todayhyperactivity_5,todayimpulsivity_1,todayimpulsivity_2,todayimpulsivity_3,todayimpulsivity_4,todaysct_1,todaysct_2,todaysct_3,todaysct_4,todaysct_5,todaysct_6,todaysct_7,todaysct_8,todaysct_9,today_na1,todaygad_1,todaygad_2,todaygad_3,todaygad_4,...,gad_3_recontact,gad_4_recontact,gad_5_recontact,gad_6_recontact,gad_7_recontact,phq_1_recontact,phq_2_recontact,phq_3_recontact,phq_4_recontact,phq_5_recontact,phq_6_recontact,phq_7_recontact,phq_8_recontact,hitop157_recontact,hitop81_recontact,hitop34_recontact,hitop54_recontact,hitop243_recontact,hitop182_recontact,hitop69_recontact,hitop89_recontact,hitop50_recontact,check_moderately_recontact,hitop129_recontact,hitop265_recontact,hitop124_recontact,hitop231_recontact,hitop93_recontact,hitop67_recontact,hitop245_recontact,hitop281_recontact,hitop141_recontact,hitop40_recontact,hitop204_recontact,hitop21_recontact,hitop236_recontact,hitop280_recontact,hitop84_recontact,hitop120_recontact,hitop77_recontact,hitop92_recontact,hitop258_recontact,hitop39_recontact,hitop254_recontact,hitop215_recontact,hitop95_recontact,hitop106_recontact,hitop283_recontact,hitop16_recontact,hitop20_recontact,hitop189_recontact,hitop1_recontact,hitop136_recontact,hitop246_recontact,hitop248_recontact,hitop257_recontact,hitop114_recontact,hitop117_recontact,hitop250_recontact,hitop200_recontact,hitop160_recontact,hitop23_recontact,hitop165_recontact,hitop244_recontact,hitop9_recontact,hitop142_recontact,hitop230_recontact,hitop149_r

In [498]:
# clean attention diagnosis - recontact
dat['attention_dx_recontact'] = ''
dat['n_attention_dx_recontact'] = (dat.loc[:, ['FNM_Q35_m_1_recontact',
                                   'FNM_Q35_m_2_recontact',
                                  ]]==1).sum(1)
dat.loc[dat.FNM_Q35_m_3_recontact == 1, 'attention_dx_recontact'] = 'other'
dat.loc[dat.FNM_Q35_m_1_recontact == 1, 'attention_dx_recontact'] = 'adhd'
dat.loc[dat.FNM_Q35_m_2_recontact == 1, 'attention_dx_recontact'] = 'adhd'

# clean other attention columns - recontact
dat.loc[dat.attention_years_recontact == 999, 'attention_years_recontact'] = np.nan
dat['attention_bothered_recontact']=False
dat.loc[dat.attention_bothered_recontact_orig == 1, 'attention_bothered_recontact'] = True

dat

,caseid,weight,sample_type,screener_1ongoing,screener_2impact,screener_3depression,screener_4anxiety,screener_5attention,consent,mood_yn,FNM_Q25_1,FNM_Q25_2,FNM_Q25_3,FNM_Q25_4,FNM_Q25_5,FNM_Q25_6,FNM_Q25_955,FNM_Q25_933,FNM_Q25_open1,mood_years,FNM_Q27_1,FNM_Q27_2,FNM_Q27_3,FNM_Q27_4,FNM_Q27_5,FNM_Q27_6,FNM_Q27_7,FNM_Q27_8,FNM_Q27_9,FNM_Q27_10,FNM_Q27_11,FNM_Q27_12,FNM_Q27_13,FNM_Q27_14,FNM_Q27_15,FNM_Q27_16,FNM_Q27_17,FNM_Q27_18,FNM_Q27_19,FNM_Q27_955,FNM_Q27_933,FNM_Q27_open1,mood_bothered_orig,anxiety_yn,FNM_Q30_m_1,FNM_Q30_m_2,FNM_Q30_m_3,FNM_Q30_m_4,FNM_Q30_m_5,FNM_Q30_m_6,FNM_Q30_m_7,FNM_Q30_m_8,FNM_Q30_m_955,FNM_Q30_m_933,FNM_Q30_open1,anxiety_years,FNM_Q32_1,FNM_Q32_2,FNM_Q32_3,FNM_Q32_4,FNM_Q32_5,FNM_Q32_6,FNM_Q32_7,FNM_Q32_8,FNM_Q32_9,FNM_Q32_10,FNM_Q32_955,FNM_Q32_933,FNM_Q32_open1,anxiety_bothered_orig,attention_yn,FNM_Q35_m_1,FNM_Q35_m_2,FNM_Q35_m_3,FNM_Q35_m_933,FNM_Q35_open1,attention_years,FNM_Q37_m_1,FNM_Q37_m_2,FNM_Q37_m_3,FNM_Q37_m_4,FNM_Q37_m_5,FNM_Q37_m_6,FNM_Q37_m_7,FNM_Q37_m_8,FNM_Q37_m_9,FNM_Q37_m_955,FNM_Q37_m_933,FNM_Q37_open1,attention_bothered_orig,inattention_1,inattention_2,inattention_3,inattention_4,inattention_5,inattention_6,FNM_Q1_attn,inattention_7,inattention_8,inattention_9,hyperactivity_1,hyperactivity_2,hyperactivity_3,hyperactivity_4,hyperactivity_5,impulsivity_1,impulsivity_2,impulsivity_3,impulsivity_4,sct_1,sct_2,sct_3,sct_4,sct_5,sct_6,sct_7,sct_8,sct_9,gad_1,gad_2,gad_3,gad_4,gad_5,gad_6,gad_7,phq_1,phq_2,phq_3,phq_4,phq_5,phq_6,phq_7,phq_8,hitop157,hitop81,hitop34,hitop54,hitop243,hitop182,hitop69,hitop89,hitop50,check_moderately,hitop129,hitop265,hitop124,hitop231,hitop93,hitop67,hitop245,hitop281,hitop141,hitop40,hitop204,hitop21,hitop236,hitop280,hitop84,hitop120,hitop77,hitop92,hitop258,hitop39,hitop254,hitop215,hitop95,hitop106,hitop283,hitop16,hitop20,hitop189,hitop1,hitop136,hitop246,hitop248,hitop257,hitop114,hitop117,hitop250,hitop200,hitop160,hitop23,hitop165,hitop244,hitop9,hitop142,hitop230,hitop149,hitop247,hitop99,hitop66,hitop240,hitop222,hitop90,hitop113,hitop278,hitop203,hitop159,hitop123,hitop275,hitop268,hitop225,hitop143,hitop151,hitop181,hitop211,hitop17,hitop126,hitop5,hitop261,hitop220,check_notatall,hitop15,hitop72,hitop140,hitop109,hitop197,hitop104,todayinattention_1,todayinattention_2,todayinattention_3,todayinattention_4,todayinattention_5,todayinattention_6,todayinattention_7,todayinattention_8,todayinattention_9,todayhyperactivity_1,todayhyperactivity_2,todayhyperactivity_3,todayhyperactivity_4,todayhyperactivity_5,todayimpulsivity_1,todayimpulsivity_2,todayimpulsivity_3,todayimpulsivity_4,todaysct_1,todaysct_2,todaysct_3,todaysct_4,todaysct_5,todaysct_6,todaysct_7,todaysct_8,todaysct_9,today_na1,todaygad_1,todaygad_2,todaygad_3,todaygad_4,...,gad_6_recontact,gad_7_recontact,phq_1_recontact,phq_2_recontact,phq_3_recontact,phq_4_recontact,phq_5_recontact,phq_6_recontact,phq_7_recontact,phq_8_recontact,hitop157_recontact,hitop81_recontact,hitop34_recontact,hitop54_recontact,hitop243_recontact,hitop182_recontact,hitop69_recontact,hitop89_recontact,hitop50_recontact,check_moderately_recontact,hitop129_recontact,hitop265_recontact,hitop124_recontact,hitop231_recontact,hitop93_recontact,hitop67_recontact,hitop245_recontact,hitop281_recontact,hitop141_recontact,hitop40_recontact,hitop204_recontact,hitop21_recontact,hitop236_recontact,hitop280_recontact,hitop84_recontact,hitop120_recontact,hitop77_recontact,hitop92_recontact,hitop258_recontact,hitop39_recontact,hitop254_recontact,hitop215_recontact,hitop95_recontact,hitop106_recontact,hitop283_recontact,hitop16_recontact,hitop20_recontact,hitop189_recontact,hitop1_recontact,hitop136_recontact,hitop246_recontact,hitop248_recontact,hitop257_recontact,hitop114_recontact,hitop117_recontact,hitop250_recontact,hitop200_recontact,hitop160_recontact,hitop23_recontact,hitop165_recontact,hitop244_recontact,hitop9_recontact,hitop142_recontact,hitop230_recontact,hitop149_recontact,hitop247_recontact,hitop99_recontact,hi

In [499]:
# deal with attention checks

# QUESTION
# not sure what passed_grid and passed_list are

dat['passed_checks'] = True
dat['passed_grid'] = True
dat['passed_list'] = True
dat.loc[dat.check_moderately != 3, 'passed_checks'] = False
dat.loc[dat.check_notatall != 1, 'passed_checks'] = False
dat.loc[dat.check_moderately != 3, 'passed_grid'] = False
dat.loc[dat.check_notatall != 1, 'passed_grid'] = False
dat.loc[dat.todaycheck_1 != 1, 'passed_checks'] = False
dat.loc[dat.todaycheck_2 != 1, 'passed_checks'] = False
dat.loc[dat.todaycheck_1 != 1, 'passed_list'] = False
dat.loc[dat.todaycheck_2 != 1, 'passed_list'] = False

In [500]:
# deal with attention checks - recontact
dat['passed_checks_recontact'] = True
dat['passed_grid_recontact'] = True
dat['passed_list_recontact'] = True
dat.loc[dat.check_moderately_recontact != 3, 'passed_checks_recontact'] = False
dat.loc[dat.check_notatall_recontact != 1, 'passed_checks_recontact'] = False
dat.loc[dat.check_moderately_recontact != 3, 'passed_grid_recontact'] = False
dat.loc[dat.check_notatall_recontact != 1, 'passed_grid_recontact'] = False
dat.loc[dat.todaycheck_1_recontact != 1, 'passed_checks_recontact'] = False
dat.loc[dat.todaycheck_2_recontact != 1, 'passed_checks_recontact'] = False
dat.loc[dat.todaycheck_1_recontact != 1, 'passed_list_recontact'] = False
dat.loc[dat.todaycheck_2_recontact != 1, 'passed_list_recontact'] = False

In [501]:
1 - dat.loc[:, ['passed_checks', 'passed_grid', 'passed_list']].mean()

passed_checks    0.293548
passed_grid      0.016129
passed_list      0.290323
dtype: float64

In [502]:
# QUESTION
# In sample data, these results were

# passed_checks    0.36
# passed_grid      0.16
# passed_list      0.32
    
# Is this suspicious?

In [503]:
1 - dat.loc[:, ['passed_checks_recontact', 'passed_grid_recontact', 'passed_list_recontact']].mean()

passed_checks_recontact    0.412903
passed_grid_recontact      0.174194
passed_list_recontact      0.412903
dtype: float64

In [504]:
((dat.todaycheck_1 == 1) | (dat.todaycheck_2==1)).mean()

0.8387096774193549

In [505]:
((dat.todaycheck_1_recontact == 1) | (dat.todaycheck_2_recontact==1)).mean()

0.7225806451612903

In [506]:
dat.sct_1_recontact

0      1
1     -1
2      4
3      2
4     -1
      ..
305   -1
306    4
307    2
308    1
309    2
Name: sct_1_recontact, Length: 310, dtype: int64

In [507]:
minus_1_cols = []
m1_stems = ['inattention', 'hyperactivity', 'impulsivity', 'sct', 'gad', 'phq', 'hitop',
           'inattention_recontact', 'hyperactivity_recontact', 'impulsivity_recontact', 'sct_recontact', 'gad_recontact', 'phq_recontact', 'hitop_recontact']
for ms in m1_stems:
    print(ms)
    if "recontact" not in ms:
        cols = list(dat.columns[dat.columns.str.contains(ms) & ~dat.columns.str.contains('today') & ~dat.columns.str.contains('sum') & ~dat.columns.str.contains('recontact')].values)
    else:
        cols = list(dat.columns[dat.columns.str.contains(ms[:-10]) & ~dat.columns.str.contains('today') & ~dat.columns.str.contains('sum') & dat.columns.str.contains('recontact')].values)
    print(cols)
    minus_1_cols.extend(cols)

inattention
['inattention_1', 'inattention_2', 'inattention_3', 'inattention_4', 'inattention_5', 'inattention_6', 'inattention_7', 'inattention_8', 'inattention_9']
hyperactivity
['hyperactivity_1', 'hyperactivity_2', 'hyperactivity_3', 'hyperactivity_4', 'hyperactivity_5']
impulsivity
['impulsivity_1', 'impulsivity_2', 'impulsivity_3', 'impulsivity_4']
sct
['sct_1', 'sct_2', 'sct_3', 'sct_4', 'sct_5', 'sct_6', 'sct_7', 'sct_8', 'sct_9']
gad
['gad_1', 'gad_2', 'gad_3', 'gad_4', 'gad_5', 'gad_6', 'gad_7']
phq
['phq_1', 'phq_2', 'phq_3', 'phq_4', 'phq_5', 'phq_6', 'phq_7', 'phq_8']
hitop
['hitop157', 'hitop81', 'hitop34', 'hitop54', 'hitop243', 'hitop182', 'hitop69', 'hitop89', 'hitop50', 'hitop129', 'hitop265', 'hitop124', 'hitop231', 'hitop93', 'hitop67', 'hitop245', 'hitop281', 'hitop141', 'hitop40', 'hitop204', 'hitop21', 'hitop236', 'hitop280', 'hitop84', 'hitop120', 'hitop77', 'hitop92', 'hitop258', 'hitop39', 'hitop254', 'hitop215', 'hitop95', 'hitop106', 'hitop283', 'hitop16', '

In [508]:
# subtract 1 from responses
dat.loc[:, minus_1_cols] -= 1

In [509]:
minus_1_cols

['inattention_1',
 'inattention_2',
 'inattention_3',
 'inattention_4',
 'inattention_5',
 'inattention_6',
 'inattention_7',
 'inattention_8',
 'inattention_9',
 'hyperactivity_1',
 'hyperactivity_2',
 'hyperactivity_3',
 'hyperactivity_4',
 'hyperactivity_5',
 'impulsivity_1',
 'impulsivity_2',
 'impulsivity_3',
 'impulsivity_4',
 'sct_1',
 'sct_2',
 'sct_3',
 'sct_4',
 'sct_5',
 'sct_6',
 'sct_7',
 'sct_8',
 'sct_9',
 'gad_1',
 'gad_2',
 'gad_3',
 'gad_4',
 'gad_5',
 'gad_6',
 'gad_7',
 'phq_1',
 'phq_2',
 'phq_3',
 'phq_4',
 'phq_5',
 'phq_6',
 'phq_7',
 'phq_8',
 'hitop157',
 'hitop81',
 'hitop34',
 'hitop54',
 'hitop243',
 'hitop182',
 'hitop69',
 'hitop89',
 'hitop50',
 'hitop129',
 'hitop265',
 'hitop124',
 'hitop231',
 'hitop93',
 'hitop67',
 'hitop245',
 'hitop281',
 'hitop141',
 'hitop40',
 'hitop204',
 'hitop21',
 'hitop236',
 'hitop280',
 'hitop84',
 'hitop120',
 'hitop77',
 'hitop92',
 'hitop258',
 'hitop39',
 'hitop254',
 'hitop215',
 'hitop95',
 'hitop106',
 'hitop283',

In [510]:
#. Not recontact
scale_lut

{'anhedonic_depression': array(['hitop157', 'hitop182', 'hitop93', 'hitop84', 'hitop77', 'hitop92',
        'hitop39', 'hitop246', 'hitop230', 'hitop123'], dtype=object),
 'anhedonic_depression_recontact': array(['hitop157_recontact', 'hitop182_recontact', 'hitop93_recontact',
        'hitop84_recontact', 'hitop77_recontact', 'hitop92_recontact',
        'hitop39_recontact', 'hitop246_recontact', 'hitop230_recontact',
        'hitop123_recontact'], dtype=object),
 'anxious_worry': array(['hitop34', 'hitop89', 'hitop265', 'hitop20', 'hitop248',
        'hitop240', 'hitop203'], dtype=object),
 'anxious_worry_recontact': array(['hitop34_recontact', 'hitop89_recontact', 'hitop265_recontact',
        'hitop20_recontact', 'hitop248_recontact', 'hitop240_recontact',
        'hitop203_recontact'], dtype=object),
 'appetite_gain': array(['hitop243', 'hitop141', 'hitop120', 'hitop275'], dtype=object),
 'appetite_gain_recontact': array(['hitop243_recontact', 'hitop141_recontact', 'hitop120_recont

In [511]:
# Summing up the values

In [512]:
m1_stems

['inattention',
 'hyperactivity',
 'impulsivity',
 'sct',
 'gad',
 'phq',
 'hitop',
 'inattention_recontact',
 'hyperactivity_recontact',
 'impulsivity_recontact',
 'sct_recontact',
 'gad_recontact',
 'phq_recontact',
 'hitop_recontact']

In [513]:
sum_cols = []

for ms in m1_stems:
    if 'hitop' not in ms:
        #print (ms)
        if "recontact" not in ms:
            cols = list(dat.columns[dat.columns.str.contains(ms) & ~dat.columns.str.contains('today') & ~dat.columns.str.contains('sum') & ~dat.columns.str.contains('recontact')].values)
            dat[f'{ms}_sum'] = dat.loc[:, cols].sum(1)
            sum_cols.append(ms + '_sum')
        else:
            cols = list(dat.columns[dat.columns.str.contains(ms[:-10]) & ~dat.columns.str.contains('today') & ~dat.columns.str.contains('sum') & dat.columns.str.contains('recontact')].values)
            dat[f'{ms[:-10]}_sum_recontact'] = dat.loc[:, cols].sum(1)
            sum_cols.append(ms[:-10] + '_sum_recontact')

print ('\n')

hitop_sums = []
for scale_name, items in scale_lut.items():
    #print(scale_name)
    #print(items)
    #print (dat.loc[:, items].sum(1))
    dat[scale_name] = dat.loc[:, items].sum(1) # adding a dat[scale_name] with a sum of all values ("items)"
    hitop_sums.append(scale_name)


In [514]:
hitop_sums

['anhedonic_depression',
 'anhedonic_depression_recontact',
 'anxious_worry',
 'anxious_worry_recontact',
 'appetite_gain',
 'appetite_gain_recontact',
 'appetite_loss',
 'appetite_loss_recontact',
 'cognitive_problems',
 'cognitive_problems_recontact',
 'hyposomnia',
 'hyposomnia_recontact',
 'indecisiveness',
 'indecisiveness_recontact',
 'insomnia',
 'insomnia_recontact',
 'panic',
 'panic_recontact',
 'separation_insecurity',
 'separation_insecurity_recontact',
 'shame_guilt',
 'shame_guilt_recontact',
 'situational_phobia',
 'situational_phobia_recontact',
 'social_anxiety',
 'social_anxiety_recontact',
 'well_being',
 'well_being_recontact']

In [515]:
"""# check
ms = 'inattention'
cols_initial = list(dat.columns[dat.columns.str.contains(ms) & ~dat.columns.str.contains('today') & ~dat.columns.str.contains('sum') & ~dat.columns.str.contains('recontact')].values)
print (cols_initial)
cols_recontact = list(dat.columns[dat.columns.str.contains(ms) & ~dat.columns.str.contains('today') & ~dat.columns.str.contains('sum') & dat.columns.str.contains('recontact')].values)
print (cols_recontact)
#dat[f'{ms}_sum'] = dat.loc[:, cols].sum(1)
#sum_cols.append(ms + '_sum')"""


"# check\nms = 'inattention'\ncols_initial = list(dat.columns[dat.columns.str.contains(ms) & ~dat.columns.str.contains('today') & ~dat.columns.str.contains('sum') & ~dat.columns.str.contains('recontact')].values)\nprint (cols_initial)\ncols_recontact = list(dat.columns[dat.columns.str.contains(ms) & ~dat.columns.str.contains('today') & ~dat.columns.str.contains('sum') & dat.columns.str.contains('recontact')].values)\nprint (cols_recontact)\n#dat[f'{ms}_sum'] = dat.loc[:, cols].sum(1)\n#sum_cols.append(ms + '_sum')"

In [516]:
sum_cols

['inattention_sum',
 'hyperactivity_sum',
 'impulsivity_sum',
 'sct_sum',
 'gad_sum',
 'phq_sum',
 'inattention_sum_recontact',
 'hyperactivity_sum_recontact',
 'impulsivity_sum_recontact',
 'sct_sum_recontact',
 'gad_sum_recontact',
 'phq_sum_recontact']

In [517]:
hitop_sums

['anhedonic_depression',
 'anhedonic_depression_recontact',
 'anxious_worry',
 'anxious_worry_recontact',
 'appetite_gain',
 'appetite_gain_recontact',
 'appetite_loss',
 'appetite_loss_recontact',
 'cognitive_problems',
 'cognitive_problems_recontact',
 'hyposomnia',
 'hyposomnia_recontact',
 'indecisiveness',
 'indecisiveness_recontact',
 'insomnia',
 'insomnia_recontact',
 'panic',
 'panic_recontact',
 'separation_insecurity',
 'separation_insecurity_recontact',
 'shame_guilt',
 'shame_guilt_recontact',
 'situational_phobia',
 'situational_phobia_recontact',
 'social_anxiety',
 'social_anxiety_recontact',
 'well_being',
 'well_being_recontact']

In [518]:
scale_lut

{'anhedonic_depression': array(['hitop157', 'hitop182', 'hitop93', 'hitop84', 'hitop77', 'hitop92',
        'hitop39', 'hitop246', 'hitop230', 'hitop123'], dtype=object),
 'anhedonic_depression_recontact': array(['hitop157_recontact', 'hitop182_recontact', 'hitop93_recontact',
        'hitop84_recontact', 'hitop77_recontact', 'hitop92_recontact',
        'hitop39_recontact', 'hitop246_recontact', 'hitop230_recontact',
        'hitop123_recontact'], dtype=object),
 'anxious_worry': array(['hitop34', 'hitop89', 'hitop265', 'hitop20', 'hitop248',
        'hitop240', 'hitop203'], dtype=object),
 'anxious_worry_recontact': array(['hitop34_recontact', 'hitop89_recontact', 'hitop265_recontact',
        'hitop20_recontact', 'hitop248_recontact', 'hitop240_recontact',
        'hitop203_recontact'], dtype=object),
 'appetite_gain': array(['hitop243', 'hitop141', 'hitop120', 'hitop275'], dtype=object),
 'appetite_gain_recontact': array(['hitop243_recontact', 'hitop141_recontact', 'hitop120_recont

In [519]:
sum = 0
for scale_name, items in scale_lut.items():
    if "well_being" not in scale_name:
        sum += len(items)

In [520]:
for scale_name, items in scale_lut.items():
    if "well_being" not in scale_name:
        print(scale_name)

anhedonic_depression
anhedonic_depression_recontact
anxious_worry
anxious_worry_recontact
appetite_gain
appetite_gain_recontact
appetite_loss
appetite_loss_recontact
cognitive_problems
cognitive_problems_recontact
hyposomnia
hyposomnia_recontact
indecisiveness
indecisiveness_recontact
insomnia
insomnia_recontact
panic
panic_recontact
separation_insecurity
separation_insecurity_recontact
shame_guilt
shame_guilt_recontact
situational_phobia
situational_phobia_recontact
social_anxiety
social_anxiety_recontact


In [521]:
sum

146

In [522]:
hitop_sums[::2][:-1] # odd indices except the last one (well-being)

['anhedonic_depression',
 'anxious_worry',
 'appetite_gain',
 'appetite_loss',
 'cognitive_problems',
 'hyposomnia',
 'indecisiveness',
 'insomnia',
 'panic',
 'separation_insecurity',
 'shame_guilt',
 'situational_phobia',
 'social_anxiety']

In [523]:
hitop_sums[1::2][:-1] # even indices except the last one (well-being)

['anhedonic_depression_recontact',
 'anxious_worry_recontact',
 'appetite_gain_recontact',
 'appetite_loss_recontact',
 'cognitive_problems_recontact',
 'hyposomnia_recontact',
 'indecisiveness_recontact',
 'insomnia_recontact',
 'panic_recontact',
 'separation_insecurity_recontact',
 'shame_guilt_recontact',
 'situational_phobia_recontact',
 'social_anxiety_recontact']

In [524]:
dat.inattention_sum_recontact

0      13
1     -18
2      22
3       2
4     -18
       ..
305   -18
306    23
307     4
308     9
309    18
Name: inattention_sum_recontact, Length: 310, dtype: int64

In [525]:
dat.inattention_sum_recontact.value_counts()

inattention_sum_recontact
-18    51
 14    16
 11    15
 12    15
 9     15
 16    13
 22    13
 7     13
 10    13
 8     13
 5     13
 18    12
 13    12
 19    12
 15    11
 17     9
 21     9
 6      9
 4      8
 3      8
 20     7
 23     6
 2      5
 24     5
 27     2
 0      2
 1      2
 26     1
Name: count, dtype: int64

In [526]:
dat.sct_sum.value_counts()

sct_sum
9     20
18    19
13    19
6     18
15    18
17    16
14    16
11    16
19    16
16    15
8     15
7     15
12    15
10    12
22    12
5     10
21     8
25     8
4      8
20     6
3      6
1      5
26     4
27     3
2      3
23     3
24     3
0      1
Name: count, dtype: int64

In [527]:
# troubleshooting the cell below
hitop_sums[1::2][:-1]

['anhedonic_depression_recontact',
 'anxious_worry_recontact',
 'appetite_gain_recontact',
 'appetite_loss_recontact',
 'cognitive_problems_recontact',
 'hyposomnia_recontact',
 'indecisiveness_recontact',
 'insomnia_recontact',
 'panic_recontact',
 'separation_insecurity_recontact',
 'shame_guilt_recontact',
 'situational_phobia_recontact',
 'social_anxiety_recontact']

In [528]:
dat['hitop_sum'] = dat.loc[:, hitop_sums[::2][:-1]].sum(1)  # [:-1] because we don't include the well-being scale
dat['hitop_sum_recontact'] = dat.loc[:, hitop_sums[1::2][:-1]].sum(1)
dat['baars_sum'] = dat.inattention_sum + dat.hyperactivity_sum + dat.impulsivity_sum
dat['baars_sum_recontact'] = dat.inattention_sum_recontact + dat.hyperactivity_sum_recontact + dat.impulsivity_sum_recontact

dat['moodanxiety_bothered'] = dat.mood_bothered | dat.anxiety_bothered
dat['moodanxiety_bothered_recontact'] = dat.mood_bothered_recontact | dat.anxiety_bothered_recontact

In [529]:
dat.phq_sum

0      14
1      22
2      15
3      12
4       3
       ..
305    22
306    12
307    18
308     8
309    14
Name: phq_sum, Length: 310, dtype: int64

In [530]:
dat.phq_sum.value_counts().sort_index()

phq_sum
0      2
1      5
2      2
3      7
4      7
5      9
6      8
7     16
8     16
9     18
10    19
11    10
12    13
13    21
14    24
15    12
16    14
17    23
18    15
19    11
20    17
21    18
22    15
23     4
24     4
Name: count, dtype: int64

In [531]:
dat.shape

(310, 798)

In [532]:
# DEALING WITH MISSING DATA

# Only leaving the data which
# a) passed all checks
# b) doesn't have missing data in the 2nd visit

In [533]:
my_columns = []
for item in ['hitop_sum', 'baars_sum', 'phq_sum', 'gad_sum', # all sums
             'mood_bothered','anxiety_bothered', 'attention_bothered', 'moodanxiety_bothered', # bothered
             'inattention_sum', 'hyperactivity_sum', 'impulsivity_sum', 'sct_sum']: # each subscale of baars
    my_columns.append(item)
    my_columns.append(item+'_recontact')
my_columns    

['hitop_sum',
 'hitop_sum_recontact',
 'baars_sum',
 'baars_sum_recontact',
 'phq_sum',
 'phq_sum_recontact',
 'gad_sum',
 'gad_sum_recontact',
 'mood_bothered',
 'mood_bothered_recontact',
 'anxiety_bothered',
 'anxiety_bothered_recontact',
 'attention_bothered',
 'attention_bothered_recontact',
 'moodanxiety_bothered',
 'moodanxiety_bothered_recontact',
 'inattention_sum',
 'inattention_sum_recontact',
 'hyperactivity_sum',
 'hyperactivity_sum_recontact',
 'impulsivity_sum',
 'impulsivity_sum_recontact',
 'sct_sum',
 'sct_sum_recontact']

In [534]:
my_columns.extend(hitop_sums) #each subscale of hitop
my_columns

['hitop_sum',
 'hitop_sum_recontact',
 'baars_sum',
 'baars_sum_recontact',
 'phq_sum',
 'phq_sum_recontact',
 'gad_sum',
 'gad_sum_recontact',
 'mood_bothered',
 'mood_bothered_recontact',
 'anxiety_bothered',
 'anxiety_bothered_recontact',
 'attention_bothered',
 'attention_bothered_recontact',
 'moodanxiety_bothered',
 'moodanxiety_bothered_recontact',
 'inattention_sum',
 'inattention_sum_recontact',
 'hyperactivity_sum',
 'hyperactivity_sum_recontact',
 'impulsivity_sum',
 'impulsivity_sum_recontact',
 'sct_sum',
 'sct_sum_recontact',
 'anhedonic_depression',
 'anhedonic_depression_recontact',
 'anxious_worry',
 'anxious_worry_recontact',
 'appetite_gain',
 'appetite_gain_recontact',
 'appetite_loss',
 'appetite_loss_recontact',
 'cognitive_problems',
 'cognitive_problems_recontact',
 'hyposomnia',
 'hyposomnia_recontact',
 'indecisiveness',
 'indecisiveness_recontact',
 'insomnia',
 'insomnia_recontact',
 'panic',
 'panic_recontact',
 'separation_insecurity',
 'separation_insecur

In [535]:
dat.shape

(310, 798)

In [536]:
good_data = dat.loc[dat.passed_grid,my_columns]

In [537]:
good_data.shape

(305, 52)

In [538]:
# saving all data for later desriptive analysis
good_data_all = dat.loc[dat.passed_grid,:]

# let's save it 
good_data_all.to_csv(path_to_write)

In [539]:
# rename columns with phq and gad sums for cnsistency in naming
good_data = good_data.rename(columns={"gad_recontact_sum": "gad_sum_recontact", "phq_recontact_sum": "phq_sum_recontact"})

In [540]:
# rename baars subscales
good_data = good_data.rename(columns={"inattention_sum": "baars_inattention_sum", 
                          "inattention_sum_recontact": "baars_inattention_sum_recontact",
                          "hyperactivity_sum": "baars_hyperactivity_sum", 
                          "hyperactivity_sum_recontact": "baars_hyperactivity_sum_recontact",
                          "impulsivity_sum": "baars_impulsivity_sum", 
                          "impulsivity_sum_recontact": "baars_impulsivity_sum_recontact",
                          "sct_sum": "baars_sct_sum",
                          "sct_sum_recontact": "baars_sct_sum_recontact"})

In [541]:
# rename hitops subscales
rename_dict = {}
for hitop_item in hitop_sums:
    rename_dict[hitop_item] = 'hitop_' + hitop_item
good_data = good_data.rename(columns=rename_dict)

In [542]:
for col in good_data.columns:
    print(col)

hitop_sum
hitop_sum_recontact
baars_sum
baars_sum_recontact
phq_sum
phq_sum_recontact
gad_sum
gad_sum_recontact
mood_bothered
mood_bothered_recontact
anxiety_bothered
anxiety_bothered_recontact
attention_bothered
attention_bothered_recontact
moodanxiety_bothered
moodanxiety_bothered_recontact
baars_inattention_sum
baars_inattention_sum_recontact
baars_hyperactivity_sum
baars_hyperactivity_sum_recontact
baars_impulsivity_sum
baars_impulsivity_sum_recontact
baars_sct_sum
baars_sct_sum_recontact
hitop_anhedonic_depression
hitop_anhedonic_depression_recontact
hitop_anxious_worry
hitop_anxious_worry_recontact
hitop_appetite_gain
hitop_appetite_gain_recontact
hitop_appetite_loss
hitop_appetite_loss_recontact
hitop_cognitive_problems
hitop_cognitive_problems_recontact
hitop_hyposomnia
hitop_hyposomnia_recontact
hitop_indecisiveness
hitop_indecisiveness_recontact
hitop_insomnia
hitop_insomnia_recontact
hitop_panic
hitop_panic_recontact
hitop_separation_insecurity
hitop_separation_insecurity_re

In [543]:
# Alright now let's remove data where in recontact columns there are negative values.
# That means "data is missing"

In [544]:
for col in good_data.columns:
    print('\n')
    print(good_data.loc[:,col].value_counts().sort_index())



hitop_sum
7      2
17     1
18     2
19     1
22     2
      ..
183    1
186    1
195    1
199    1
206    1
Name: count, Length: 132, dtype: int64


hitop_sum_recontact
-146    48
 14      1
 17      1
 21      2
 23      1
        ..
 179     1
 182     1
 188     1
 196     1
 212     1
Name: count, Length: 116, dtype: int64


baars_sum
0      5
2      1
3      2
4      4
5      4
6      5
7      4
8      7
9     11
10     4
11     9
12     5
13    10
14     4
15     9
16    12
17    10
18    16
19     9
20     9
21    13
22     8
23    14
24    10
25    11
26     9
27    14
28     7
29     7
30    11
31    10
32     7
33     7
34     3
35     2
36     2
37     2
38     5
39     6
40     2
41     3
42     4
43     2
46     3
47     1
50     1
54     1
Name: count, dtype: int64


baars_sum_recontact
-36    48
 2      2
 4      3
 5      6
 6      3
 7      6
 8      6
 9      3
 10     7
 11     7
 12    10
 13    11
 14     5
 15     9
 16     8
 17     8
 18    10
 19    10
 20  

In [545]:
good_data = good_data[(good_data >= 0).all(axis=1)]
good_data.shape

(257, 52)

In [546]:
for col in good_data.columns:
    print('\n')
    print(good_data.loc[:,col].value_counts().sort_index())



hitop_sum
7      1
17     1
22     1
23     1
28     1
      ..
176    1
183    1
186    1
199    1
206    1
Name: count, Length: 120, dtype: int64


hitop_sum_recontact
14     1
17     1
21     2
23     1
31     3
      ..
179    1
182    1
188    1
196    1
212    1
Name: count, Length: 115, dtype: int64


baars_sum
0      3
2      1
3      2
4      3
5      2
6      5
7      3
8      7
9      7
10     2
11     7
12     4
13     7
14     4
15     7
16    10
17     8
18    14
19     6
20     7
21    11
22     8
23    13
24     8
25    11
26     9
27    11
28     5
29     7
30    11
31    10
32     6
33     7
34     2
35     2
36     1
37     2
38     5
39     6
40     2
41     2
42     3
43     1
46     2
47     1
50     1
54     1
Name: count, dtype: int64


baars_sum_recontact
2      2
4      3
5      6
6      3
7      6
8      6
9      3
10     7
11     7
12    10
13    11
14     5
15     9
16     8
17     8
18    10
19    10
20     8
21     6
22     6
23     8
24     5
25     7


In [547]:
# let's save it 
good_data.to_csv(path_to_write_ICC)